In [1]:
import pandas as pd
import csv
import os
import sys
import csv
import time
import numpy as np
import random
from scipy.io import arff

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

sys.path.insert(0, f"{os.path.dirname(os.getcwd())}/src")
from utils import evaluate_result
from data_imbalance_src.smote_oversampling import RandomOversampling, ADASYNOversampling, BorderlineSMOTEOversampling, SMOTEOversampling, SVMSMOTEOversampling
from data_imbalance_src.smote_oversampling import SMOTUNEDOversampling
from data_imbalance_src.dazzle import DAZZLEOversampling
from data_imbalance_src.dazzle1 import DAZZLE1Oversampling
from data_imbalance_src.dazzle2 import DAZZLE2Oversampling
from data_imbalance_src.Imbalance_Farou2022.data_generation import GANOversampling
from data_imbalance_src.random_projection import RandomProjectionOversampling

from diveplane.utilities import infer_feature_attributes
from diveplane.geminai import Geminai

from DataSynthesizer.DataDescriber import DataDescriber
from DataSynthesizer.DataGenerator import DataGenerator
from DataSynthesizer.ModelInspector import ModelInspector
from DataSynthesizer.lib.utils import read_json_file, display_bayesian_network

from sdv.metadata import SingleTableMetadata
from sdv.lite import SingleTablePreset
from sdv.single_table import CTGANSynthesizer
from sdv.single_table import GaussianCopulaSynthesizer

2023-10-04 15:25:40.694190: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-04 15:25:40.696474: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-04 15:25:40.731609: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-04 15:25:40.732383: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-04 15:25:41.250714: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [2]:
rs_list = random.sample(range(50, 500), 10)

In [ ]:
for repeat in range(8, 10):
    print(f"----- in repeat {repeat+1} -----")
    rs = rs_list[repeat]
    write_file = f"JS_Vuln_nodrop_res_r{repeat+1}_rn{rs}.csv"
    write_path = f"{os.path.dirname(os.getcwd())}/result/{write_file}"
    with open(write_path, "w", newline="") as f:
        csv_writer = csv.writer(f)
        csv_writer.writerow(["oversampling_scheme", "runtime", "learner", "acc", "prec", "recall", "fpr", "f1", "auc", "g_score", "d2h"])
        
    data_path = f"{os.path.dirname(os.getcwd())}/data/JavaScript_Vulnerability/"
    datafiles = [f for f in os.listdir(data_path) if f.endswith("csv")]
    
    df = pd.read_csv(f"{data_path}/{datafiles[0]}")
    drop_columns = ["name", "longname", "path", "full_repo_path", "line", "column", "endline", "endcolumn"]
    df = df.drop(drop_columns, axis=1)
#     df = df.drop_duplicates()
    df.reset_index(inplace=True, drop=True)
    
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]
    print("y value counts: \n", str(y.value_counts()))
    print("y class ratio: 1:", str(round(y.value_counts()[0]/y.value_counts()[1])))
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=rs)
    print("--- y train classes count: \n" + str(y_train.value_counts()))
    print("--- y train ratio: 1:" + str(round(y_train.value_counts()[0] / y_train.value_counts()[1])))
    print(" ")
    print("--- y test classes count: \n" + str(y_test.value_counts()))
    print("--- y test ratio: 1:" + str(round(y_test.value_counts()[0] / y_test.value_counts()[1])))
    
    ########## normal run ##########
    # normal run - without any oversampling technique
    print("----- normal -----")

    # scale data
    scaler = StandardScaler()
    X_train_scale = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns, index=X_train.index)
    X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

    # create models
    clf_SVM = SVC()
    clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
    clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
    clf_DT = DecisionTreeClassifier()
    clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
    clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
    clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
    clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

    clf_SVM.fit(X_train_scale, y_train)
    clf_KNN.fit(X_train_scale, y_train)
    clf_LR.fit(X_train_scale, y_train)
    clf_DT.fit(X_train_scale, y_train)
    clf_RF.fit(X_train_scale, y_train)
    clf_LightGBM.fit(X_train_scale, y_train)
    clf_Adaboost.fit(X_train_scale, y_train)
    clf_GBDT.fit(X_train_scale, y_train)

    y_pred_SVM = clf_SVM.predict(X_test_scale)
    y_pred_KNN = clf_KNN.predict(X_test_scale)
    y_pred_LR = clf_LR.predict(X_test_scale)
    y_pred_DT = clf_DT.predict(X_test_scale)
    y_pred_RF = clf_RF.predict(X_test_scale)
    y_pred_LightGBM = clf_LightGBM.predict(X_test_scale)
    y_pred_Adaboost = clf_Adaboost.predict(X_test_scale)
    y_pred_GBDT = clf_GBDT.predict(X_test_scale)
    
    with open(write_path, "a", newline="") as f:
        csv_writer = csv.writer(f)

        csv_writer.writerow(["No", 0, "SVM"] + evaluate_result(y_pred_SVM, y_test))
        csv_writer.writerow(["No", 0, "KNN"] + evaluate_result(y_pred_KNN, y_test))
        csv_writer.writerow(["No", 0, "LR"] + evaluate_result(y_pred_LR, y_test))
        csv_writer.writerow(["No", 0, "DT"] + evaluate_result(y_pred_DT, y_test))
        csv_writer.writerow(["No", 0, "RF"] + evaluate_result(y_pred_RF, y_test))
        csv_writer.writerow(["No", 0, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
        csv_writer.writerow(["No", 0, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
        csv_writer.writerow(["No", 0, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))
    
    ########## random run ##########
    # random oversampling run - random oversampling technique
    print("----- random -----")

    rt, X_train_new, y_train_new = RandomOversampling(X_train=X_train, y_train=y_train)

    # scale data
    scaler = StandardScaler()
    X_train_scale = pd.DataFrame(scaler.fit_transform(X_train_new), columns=X_train_new.columns, index=X_train_new.index)
    X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

    print("y train ratio: 1:" + str(round(y_train_new.value_counts()[0] / y_train_new.value_counts()[1])))

    # create models
    clf_SVM = SVC()
    clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
    clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
    clf_DT = DecisionTreeClassifier()
    clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
    clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
    clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
    clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

    clf_SVM.fit(X_train_scale, y_train_new)
    clf_KNN.fit(X_train_scale, y_train_new)
    clf_LR.fit(X_train_scale, y_train_new)
    clf_DT.fit(X_train_scale, y_train_new)
    clf_RF.fit(X_train_scale, y_train_new)
    clf_LightGBM.fit(X_train_scale, y_train_new)
    clf_Adaboost.fit(X_train_scale, y_train_new)
    clf_GBDT.fit(X_train_scale, y_train_new)

    y_pred_SVM = clf_SVM.predict(X_test_scale)
    y_pred_KNN = clf_KNN.predict(X_test_scale)
    y_pred_LR = clf_LR.predict(X_test_scale)
    y_pred_DT = clf_DT.predict(X_test_scale)
    y_pred_RF = clf_RF.predict(X_test_scale)
    y_pred_LightGBM = clf_LightGBM.predict(X_test_scale)
    y_pred_Adaboost = clf_Adaboost.predict(X_test_scale)
    y_pred_GBDT = clf_GBDT.predict(X_test_scale)
    
    with open(write_path, "a", newline="") as f:
        csv_writer = csv.writer(f)

        csv_writer.writerow(["Random", rt, "SVM"] + evaluate_result(y_pred_SVM, y_test))
        csv_writer.writerow(["Random", rt, "KNN"] + evaluate_result(y_pred_KNN, y_test))
        csv_writer.writerow(["Random", rt, "LR"] + evaluate_result(y_pred_LR, y_test))
        csv_writer.writerow(["Random", rt, "DT"] + evaluate_result(y_pred_DT, y_test))
        csv_writer.writerow(["Random", rt, "RF"] + evaluate_result(y_pred_RF, y_test))
        csv_writer.writerow(["Random", rt, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
        csv_writer.writerow(["Random", rt, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
        csv_writer.writerow(["Random", rt, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))
    
    ########## ADASYN run ##########
    # ADASYN oversampling run - ADASYN oversampling technique
    print("----- ADASYN ------")

    rt, X_train_new, y_train_new = ADASYNOversampling(X_train=X_train, y_train=y_train)

    # scale data
    scaler = StandardScaler()
    X_train_scale = pd.DataFrame(scaler.fit_transform(X_train_new), columns=X_train_new.columns, index=X_train_new.index)
    X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

    print("y train ratio: 1:" + str(round(y_train_new.value_counts()[0] / y_train_new.value_counts()[1])))

    # create models
    clf_SVM = SVC()
    clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
    clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
    clf_DT = DecisionTreeClassifier()
    clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
    clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
    clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
    clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

    clf_SVM.fit(X_train_scale, y_train_new)
    clf_KNN.fit(X_train_scale, y_train_new)
    clf_LR.fit(X_train_scale, y_train_new)
    clf_DT.fit(X_train_scale, y_train_new)
    clf_RF.fit(X_train_scale, y_train_new)
    clf_LightGBM.fit(X_train_scale, y_train_new)
    clf_Adaboost.fit(X_train_scale, y_train_new)
    clf_GBDT.fit(X_train_scale, y_train_new)

    y_pred_SVM = clf_SVM.predict(X_test_scale)
    y_pred_KNN = clf_KNN.predict(X_test_scale)
    y_pred_LR = clf_LR.predict(X_test_scale)
    y_pred_DT = clf_DT.predict(X_test_scale)
    y_pred_RF = clf_RF.predict(X_test_scale)
    y_pred_LightGBM = clf_LightGBM.predict(X_test_scale)
    y_pred_Adaboost = clf_Adaboost.predict(X_test_scale)
    y_pred_GBDT = clf_GBDT.predict(X_test_scale)
    
    with open(write_path, "a", newline="") as f:
        csv_writer = csv.writer(f)

        csv_writer.writerow(["ADASYN", rt, "SVM"] + evaluate_result(y_pred_SVM, y_test))
        csv_writer.writerow(["ADASYN", rt, "KNN"] + evaluate_result(y_pred_KNN, y_test))
        csv_writer.writerow(["ADASYN", rt, "LR"] + evaluate_result(y_pred_LR, y_test))
        csv_writer.writerow(["ADASYN", rt, "DT"] + evaluate_result(y_pred_DT, y_test))
        csv_writer.writerow(["ADASYN", rt, "RF"] + evaluate_result(y_pred_RF, y_test))
        csv_writer.writerow(["ADASYN", rt, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
        csv_writer.writerow(["ADASYN", rt, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
        csv_writer.writerow(["ADASYN", rt, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))
    
    ########## BorderlineSMOTE run ##########
    # BorderlineSMOTE oversampling run - BorderlineSMOTE oversampling technique
    print("----- borderlineSMOTE -----")

    rt, X_train_new, y_train_new = BorderlineSMOTEOversampling(X_train=X_train, y_train=y_train)

    # scale data
    scaler = StandardScaler()
    X_train_scale = pd.DataFrame(scaler.fit_transform(X_train_new), columns=X_train_new.columns, index=X_train_new.index)
    X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

    print("y train ratio: 1:" + str(round(y_train_new.value_counts()[0] / y_train_new.value_counts()[1])))

    # create models
    clf_SVM = SVC()
    clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
    clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
    clf_DT = DecisionTreeClassifier()
    clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
    clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
    clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
    clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

    clf_SVM.fit(X_train_scale, y_train_new)
    clf_KNN.fit(X_train_scale, y_train_new)
    clf_LR.fit(X_train_scale, y_train_new)
    clf_DT.fit(X_train_scale, y_train_new)
    clf_RF.fit(X_train_scale, y_train_new)
    clf_LightGBM.fit(X_train_scale, y_train_new)
    clf_Adaboost.fit(X_train_scale, y_train_new)
    clf_GBDT.fit(X_train_scale, y_train_new)

    y_pred_SVM = clf_SVM.predict(X_test_scale)
    y_pred_KNN = clf_KNN.predict(X_test_scale)
    y_pred_LR = clf_LR.predict(X_test_scale)
    y_pred_DT = clf_DT.predict(X_test_scale)
    y_pred_RF = clf_RF.predict(X_test_scale)
    y_pred_LightGBM = clf_LightGBM.predict(X_test_scale)
    y_pred_Adaboost = clf_Adaboost.predict(X_test_scale)
    y_pred_GBDT = clf_GBDT.predict(X_test_scale)
    
    with open(write_path, "a", newline="") as f:
        csv_writer = csv.writer(f)

        csv_writer.writerow(["BorderlineSMOTE", rt, "SVM"] + evaluate_result(y_pred_SVM, y_test))
        csv_writer.writerow(["BorderlineSMOTE", rt, "KNN"] + evaluate_result(y_pred_KNN, y_test))
        csv_writer.writerow(["BorderlineSMOTE", rt, "LR"] + evaluate_result(y_pred_LR, y_test))
        csv_writer.writerow(["BorderlineSMOTE", rt, "DT"] + evaluate_result(y_pred_DT, y_test))
        csv_writer.writerow(["BorderlineSMOTE", rt, "RF"] + evaluate_result(y_pred_RF, y_test))
        csv_writer.writerow(["BorderlineSMOTE", rt, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
        csv_writer.writerow(["BorderlineSMOTE", rt, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
        csv_writer.writerow(["BorderlineSMOTE", rt, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))
    
    ########## SMOTE run ##########
    # SMOTE oversampling run - SMOTE oversampling technique
    print("----- SMOTE -----")

    rt, X_train_new, y_train_new = SMOTEOversampling(X_train=X_train, y_train=y_train)

    # scale data
    scaler = StandardScaler()
    X_train_scale = pd.DataFrame(scaler.fit_transform(X_train_new), columns=X_train_new.columns, index=X_train_new.index)
    X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

    print("y train ratio: 1:" + str(round(y_train_new.value_counts()[0] / y_train_new.value_counts()[1])))

    # create models
    clf_SVM = SVC()
    clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
    clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
    clf_DT = DecisionTreeClassifier()
    clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
    clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
    clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
    clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

    clf_SVM.fit(X_train_scale, y_train_new)
    clf_KNN.fit(X_train_scale, y_train_new)
    clf_LR.fit(X_train_scale, y_train_new)
    clf_DT.fit(X_train_scale, y_train_new)
    clf_RF.fit(X_train_scale, y_train_new)
    clf_LightGBM.fit(X_train_scale, y_train_new)
    clf_Adaboost.fit(X_train_scale, y_train_new)
    clf_GBDT.fit(X_train_scale, y_train_new)

    y_pred_SVM = clf_SVM.predict(X_test_scale)
    y_pred_KNN = clf_KNN.predict(X_test_scale)
    y_pred_LR = clf_LR.predict(X_test_scale)
    y_pred_DT = clf_DT.predict(X_test_scale)
    y_pred_RF = clf_RF.predict(X_test_scale)
    y_pred_LightGBM = clf_LightGBM.predict(X_test_scale)
    y_pred_Adaboost = clf_Adaboost.predict(X_test_scale)
    y_pred_GBDT = clf_GBDT.predict(X_test_scale)
    
    with open(write_path, "a", newline="") as f:
        csv_writer = csv.writer(f)

        csv_writer.writerow(["SMOTE", rt, "SVM"] + evaluate_result(y_pred_SVM, y_test))
        csv_writer.writerow(["SMOTE", rt, "KNN"] + evaluate_result(y_pred_KNN, y_test))
        csv_writer.writerow(["SMOTE", rt, "LR"] + evaluate_result(y_pred_LR, y_test))
        csv_writer.writerow(["SMOTE", rt, "DT"] + evaluate_result(y_pred_DT, y_test))
        csv_writer.writerow(["SMOTE", rt, "RF"] + evaluate_result(y_pred_RF, y_test))
        csv_writer.writerow(["SMOTE", rt, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
        csv_writer.writerow(["SMOTE", rt, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
        csv_writer.writerow(["SMOTE", rt, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))
    
    ########## SVMSMOTE run ##########
    # SVMSMOTE oversampling run - SVMSMOTE oversampling technique
    print("----- SVMSMOTE -----")

    rt, X_train_new, y_train_new = SVMSMOTEOversampling(X_train=X_train, y_train=y_train)

    # scale data
    scaler = StandardScaler()
    X_train_scale = pd.DataFrame(scaler.fit_transform(X_train_new), columns=X_train_new.columns, index=X_train_new.index)
    X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

    print("y train ratio: 1:" + str(round(y_train_new.value_counts()[0] / y_train_new.value_counts()[1])))

    # create models
    clf_SVM = SVC()
    clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
    clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
    clf_DT = DecisionTreeClassifier()
    clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
    clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
    clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
    clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

    clf_SVM.fit(X_train_scale, y_train_new)
    clf_KNN.fit(X_train_scale, y_train_new)
    clf_LR.fit(X_train_scale, y_train_new)
    clf_DT.fit(X_train_scale, y_train_new)
    clf_RF.fit(X_train_scale, y_train_new)
    clf_LightGBM.fit(X_train_scale, y_train_new)
    clf_Adaboost.fit(X_train_scale, y_train_new)
    clf_GBDT.fit(X_train_scale, y_train_new)

    y_pred_SVM = clf_SVM.predict(X_test_scale)
    y_pred_KNN = clf_KNN.predict(X_test_scale)
    y_pred_LR = clf_LR.predict(X_test_scale)
    y_pred_DT = clf_DT.predict(X_test_scale)
    y_pred_RF = clf_RF.predict(X_test_scale)
    y_pred_LightGBM = clf_LightGBM.predict(X_test_scale)
    y_pred_Adaboost = clf_Adaboost.predict(X_test_scale)
    y_pred_GBDT = clf_GBDT.predict(X_test_scale)
    
    with open(write_path, "a", newline="") as f:
        csv_writer = csv.writer(f)

        csv_writer.writerow(["SVMSMOTE", rt, "SVM"] + evaluate_result(y_pred_SVM, y_test))
        csv_writer.writerow(["SVMSMOTE", rt, "KNN"] + evaluate_result(y_pred_KNN, y_test))
        csv_writer.writerow(["SVMSMOTE", rt, "LR"] + evaluate_result(y_pred_LR, y_test))
        csv_writer.writerow(["SVMSMOTE", rt, "DT"] + evaluate_result(y_pred_DT, y_test))
        csv_writer.writerow(["SVMSMOTE", rt, "RF"] + evaluate_result(y_pred_RF, y_test))
        csv_writer.writerow(["SVMSMOTE", rt, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
        csv_writer.writerow(["SVMSMOTE", rt, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
        csv_writer.writerow(["SVMSMOTE", rt, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))
    
    ########## SMOTUNED run ##########
    # SMOTUNED oversampling run - SMOTUNED oversampling technique
    print("----- SMOTUNED -----")

    rt_SVM, X_train_new_SVM, y_train_new_SVM = SMOTUNEDOversampling(X_train=X_train, X_test=X_test, 
                                                                    y_train=y_train, y_test=y_test, model="SVM")

    scaler = StandardScaler()
    X_train_scale_SVM = pd.DataFrame(scaler.fit_transform(X_train_new_SVM), columns=X_train_new_SVM.columns)
    X_test_scale_SVM = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

    print("y train ratio of SVM: 1:" + str(round(y_train_new_SVM.value_counts()[0] / y_train_new_SVM.value_counts()[1])))

    rt_KNN, X_train_new_KNN, y_train_new_KNN = SMOTUNEDOversampling(X_train=X_train, X_test=X_test, 
                                                                    y_train=y_train, y_test=y_test, model="KNN")

    scaler = StandardScaler()
    X_train_scale_KNN = pd.DataFrame(scaler.fit_transform(X_train_new_KNN), columns=X_train_new_KNN.columns)
    X_test_scale_KNN = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

    print("y train ratio of KNN: 1:" + str(round(y_train_new_KNN.value_counts()[0] / y_train_new_KNN.value_counts()[1])))

    rt_LR, X_train_new_LR, y_train_new_LR = SMOTUNEDOversampling(X_train=X_train, X_test=X_test, 
                                                                 y_train=y_train, y_test=y_test, model="LR")

    scaler = StandardScaler()
    X_train_scale_LR = pd.DataFrame(scaler.fit_transform(X_train_new_LR), columns=X_train_new_LR.columns)
    X_test_scale_LR = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

    print("y train ratio of LR: 1:" + str(round(y_train_new_LR.value_counts()[0] / y_train_new_LR.value_counts()[1])))

    rt_DT, X_train_new_DT, y_train_new_DT = SMOTUNEDOversampling(X_train=X_train, X_test=X_test, 
                                                                 y_train=y_train, y_test=y_test, model="DT")

    scaler = StandardScaler()
    X_train_scale_DT = pd.DataFrame(scaler.fit_transform(X_train_new_DT), columns=X_train_new_DT.columns)
    X_test_scale_DT = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

    print("y train ratio of DT: 1:" + str(round(y_train_new_DT.value_counts()[0] / y_train_new_DT.value_counts()[1])))

    rt_RF, X_train_new_RF, y_train_new_RF = SMOTUNEDOversampling(X_train=X_train, X_test=X_test, 
                                                                 y_train=y_train, y_test=y_test, model="RF")

    scaler = StandardScaler()
    X_train_scale_RF = pd.DataFrame(scaler.fit_transform(X_train_new_RF), columns=X_train_new_RF.columns)
    X_test_scale_RF = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

    print("y train ratio of RF: 1:" + str(round(y_train_new_RF.value_counts()[0] / y_train_new_RF.value_counts()[1])))

    rt_LightGBM, X_train_new_LightGBM, y_train_new_LightGBM = SMOTUNEDOversampling(X_train=X_train, X_test=X_test, 
                                                                                   y_train=y_train, y_test=y_test, model="LightGBM")

    scaler = StandardScaler()
    X_train_scale_LightGBM = pd.DataFrame(scaler.fit_transform(X_train_new_LightGBM), columns=X_train_new_LightGBM.columns)
    X_test_scale_LightGBM = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

    print("y train ratio of LightGBM: 1:" + str(round(y_train_new_LightGBM.value_counts()[0] / y_train_new_LightGBM.value_counts()[1])))

    rt_Adaboost, X_train_new_Adaboost, y_train_new_Adaboost = SMOTUNEDOversampling(X_train=X_train, X_test=X_test, 
                                                                                   y_train=y_train, y_test=y_test, model="Adaboost")

    scaler = StandardScaler()
    X_train_scale_Adaboost = pd.DataFrame(scaler.fit_transform(X_train_new_Adaboost), columns=X_train_new_Adaboost.columns)
    X_test_scale_Adaboost = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

    print("y train ratio of Adaboost: 1:" + str(round(y_train_new_Adaboost.value_counts()[0] / y_train_new_Adaboost.value_counts()[1])))

    rt_GBDT, X_train_new_GBDT, y_train_new_GBDT = SMOTUNEDOversampling(X_train=X_train, X_test=X_test, 
                                                                       y_train=y_train, y_test=y_test, model="GBDT")

    scaler = StandardScaler()
    X_train_scale_GBDT = pd.DataFrame(scaler.fit_transform(X_train_new_GBDT), columns=X_train_new_GBDT.columns)
    X_test_scale_GBDT = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

    print("y train ratio of GBDT: 1:" + str(round(y_train_new_GBDT.value_counts()[0] / y_train_new_GBDT.value_counts()[1])))

    # create models
    clf_SVM = SVC()
    clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
    clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
    clf_DT = DecisionTreeClassifier()
    clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
    clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
    clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
    clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

    clf_SVM.fit(X_train_scale_SVM, y_train_new_SVM)
    clf_KNN.fit(X_train_scale_KNN, y_train_new_KNN)
    clf_LR.fit(X_train_scale_LR, y_train_new_LR)
    clf_DT.fit(X_train_scale_DT, y_train_new_DT)
    clf_RF.fit(X_train_scale_RF, y_train_new_RF)
    clf_LightGBM.fit(X_train_scale_LightGBM, y_train_new_LightGBM)
    clf_Adaboost.fit(X_train_scale_Adaboost, y_train_new_Adaboost)
    clf_GBDT.fit(X_train_scale_GBDT, y_train_new_GBDT)

    y_pred_SVM = clf_SVM.predict(X_test_scale_SVM)
    y_pred_KNN = clf_KNN.predict(X_test_scale_KNN)
    y_pred_LR = clf_LR.predict(X_test_scale_LR)
    y_pred_DT = clf_DT.predict(X_test_scale_DT)
    y_pred_RF = clf_RF.predict(X_test_scale_RF)
    y_pred_LightGBM = clf_LightGBM.predict(X_test_scale_LightGBM)
    y_pred_Adaboost = clf_Adaboost.predict(X_test_scale_Adaboost)
    y_pred_GBDT = clf_GBDT.predict(X_test_scale_GBDT)
    
    with open(write_path, "a", newline="") as f:
        csv_writer = csv.writer(f)

        csv_writer.writerow(["SMOTUNED", rt_SVM, "SVM"] + evaluate_result(y_pred_SVM, y_test))
        csv_writer.writerow(["SMOTUNED", rt_KNN, "KNN"] + evaluate_result(y_pred_KNN, y_test))
        csv_writer.writerow(["SMOTUNED", rt_LR, "LR"] + evaluate_result(y_pred_LR, y_test))
        csv_writer.writerow(["SMOTUNED", rt_DT, "DT"] + evaluate_result(y_pred_DT, y_test))
        csv_writer.writerow(["SMOTUNED", rt_RF, "RF"] + evaluate_result(y_pred_RF, y_test))
        csv_writer.writerow(["SMOTUNED", rt_LightGBM, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
        csv_writer.writerow(["SMOTUNED", rt_Adaboost, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
        csv_writer.writerow(["SMOTUNED", rt_GBDT, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))
    
    ########## DAZZLE run ##########
    # DAZZLE oversampling run - DAZZLE oversampling technique
    print("----- DAZZLE -----")

    scaler = StandardScaler()
    X_train_GAN = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns, index=X_train.index)
    rt, X_train_new, y_train_new = DAZZLEOversampling(X_train=X_train_GAN, y_train=y_train)

    # scale data
    scaler = StandardScaler()
    X_train_scale = pd.DataFrame(scaler.fit_transform(X_train_new), columns=X_train_new.columns, index=X_train_new.index)
    X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

    print("y train ratio: 1:" + str(round(y_train_new.value_counts()[0] / y_train_new.value_counts()[1])))

    # create models
    clf_SVM = SVC()
    clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
    clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
    clf_DT = DecisionTreeClassifier()
    clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
    clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
    clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
    clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

    clf_SVM.fit(X_train_scale, y_train_new)
    clf_KNN.fit(X_train_scale, y_train_new)
    clf_LR.fit(X_train_scale, y_train_new)
    clf_DT.fit(X_train_scale, y_train_new)
    clf_RF.fit(X_train_scale, y_train_new)
    clf_LightGBM.fit(X_train_scale, y_train_new)
    clf_Adaboost.fit(X_train_scale, y_train_new)
    clf_GBDT.fit(X_train_scale, y_train_new)

    y_pred_SVM = clf_SVM.predict(X_test_scale)
    y_pred_KNN = clf_KNN.predict(X_test_scale)
    y_pred_LR = clf_LR.predict(X_test_scale)
    y_pred_DT = clf_DT.predict(X_test_scale)
    y_pred_RF = clf_RF.predict(X_test_scale)
    y_pred_LightGBM = clf_LightGBM.predict(X_test_scale)
    y_pred_Adaboost = clf_Adaboost.predict(X_test_scale)
    y_pred_GBDT = clf_GBDT.predict(X_test_scale)
    
    with open(write_path, "a", newline="") as f:
        csv_writer = csv.writer(f)

        csv_writer.writerow(["DAZZLE", rt, "SVM"] + evaluate_result(y_pred_SVM, y_test))
        csv_writer.writerow(["DAZZLE", rt, "KNN"] + evaluate_result(y_pred_KNN, y_test))
        csv_writer.writerow(["DAZZLE", rt, "LR"] + evaluate_result(y_pred_LR, y_test))
        csv_writer.writerow(["DAZZLE", rt, "DT"] + evaluate_result(y_pred_DT, y_test))
        csv_writer.writerow(["DAZZLE", rt, "RF"] + evaluate_result(y_pred_RF, y_test))
        csv_writer.writerow(["DAZZLE", rt, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
        csv_writer.writerow(["DAZZLE", rt, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
        csv_writer.writerow(["DAZZLE", rt, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))
    
    ########## WGAN run ##########
    # WGAN oversampling run - WGAN oversampling technique
    print("----- WGAN -----")

    scaler = StandardScaler()
    X_train_GAN = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns, index=X_train.index)
    rt, X_train_new, y_train_new = GANOversampling(X_train=X_train_GAN, y_train=y_train)

    # scale data
    scaler = StandardScaler()
    X_train_scale = pd.DataFrame(scaler.fit_transform(X_train_new), columns=X_train_new.columns, index=X_train_new.index)
    X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

    print("y train ratio: 1:" + str(round(y_train_new.value_counts()[0] / y_train_new.value_counts()[1])))

    # create models
    clf_SVM = SVC()
    clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
    clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
    clf_DT = DecisionTreeClassifier()
    clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
    clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
    clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
    clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

    clf_SVM.fit(X_train_scale, y_train_new)
    clf_KNN.fit(X_train_scale, y_train_new)
    clf_LR.fit(X_train_scale, y_train_new)
    clf_DT.fit(X_train_scale, y_train_new)
    clf_RF.fit(X_train_scale, y_train_new)
    clf_LightGBM.fit(X_train_scale, y_train_new)
    clf_Adaboost.fit(X_train_scale, y_train_new)
    clf_GBDT.fit(X_train_scale, y_train_new)

    y_pred_SVM = clf_SVM.predict(X_test_scale)
    y_pred_KNN = clf_KNN.predict(X_test_scale)
    y_pred_LR = clf_LR.predict(X_test_scale)
    y_pred_DT = clf_DT.predict(X_test_scale)
    y_pred_RF = clf_RF.predict(X_test_scale)
    y_pred_LightGBM = clf_LightGBM.predict(X_test_scale)
    y_pred_Adaboost = clf_Adaboost.predict(X_test_scale)
    y_pred_GBDT = clf_GBDT.predict(X_test_scale)
    
    with open(write_path, "a", newline="") as f:
        csv_writer = csv.writer(f)

        csv_writer.writerow(["WGAN", rt, "SVM"] + evaluate_result(y_pred_SVM, y_test))
        csv_writer.writerow(["WGAN", rt, "KNN"] + evaluate_result(y_pred_KNN, y_test))
        csv_writer.writerow(["WGAN", rt, "LR"] + evaluate_result(y_pred_LR, y_test))
        csv_writer.writerow(["WGAN", rt, "DT"] + evaluate_result(y_pred_DT, y_test))
        csv_writer.writerow(["WGAN", rt, "RF"] + evaluate_result(y_pred_RF, y_test))
        csv_writer.writerow(["WGAN", rt, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
        csv_writer.writerow(["WGAN", rt, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
        csv_writer.writerow(["WGAN", rt, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))


    ########## RandomProjection run ##########
    # Random projection oversampling run - Random projection oversampling technique
    print("----- Random Projection -----")
    X_train_RP = X_train.copy()
    y_train_RP = y_train.copy()
    rt, X_train_new, y_train_new = RandomProjectionOversampling(X_train=X_train_RP, y_train=y_train_RP)
    
    # scale data
    scaler = StandardScaler()
    X_train_scale = pd.DataFrame(scaler.fit_transform(X_train_new), columns=X_train_new.columns, index=X_train_new.index)
    X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

    print("y train ratio: 1:" + str(round(y_train_new.value_counts()[0] / y_train_new.value_counts()[1])))

    # create models
    clf_SVM = SVC()
    clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
    clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
    clf_DT = DecisionTreeClassifier()
    clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
    clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
    clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
    clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

    clf_SVM.fit(X_train_scale, y_train_new)
    clf_KNN.fit(X_train_scale, y_train_new)
    clf_LR.fit(X_train_scale, y_train_new)
    clf_DT.fit(X_train_scale, y_train_new)
    clf_RF.fit(X_train_scale, y_train_new)
    clf_LightGBM.fit(X_train_scale, y_train_new)
    clf_Adaboost.fit(X_train_scale, y_train_new)
    clf_GBDT.fit(X_train_scale, y_train_new)

    y_pred_SVM = clf_SVM.predict(X_test_scale)
    y_pred_KNN = clf_KNN.predict(X_test_scale)
    y_pred_LR = clf_LR.predict(X_test_scale)
    y_pred_DT = clf_DT.predict(X_test_scale)
    y_pred_RF = clf_RF.predict(X_test_scale)
    y_pred_LightGBM = clf_LightGBM.predict(X_test_scale)
    y_pred_Adaboost = clf_Adaboost.predict(X_test_scale)
    y_pred_GBDT = clf_GBDT.predict(X_test_scale)

    with open(write_path, "a", newline="") as f:
        csv_writer = csv.writer(f)

        csv_writer.writerow(["RP", rt, "SVM"] + evaluate_result(y_pred_SVM, y_test))
        csv_writer.writerow(["RP", rt, "KNN"] + evaluate_result(y_pred_KNN, y_test))
        csv_writer.writerow(["RP", rt, "LR"] + evaluate_result(y_pred_LR, y_test))
        csv_writer.writerow(["RP", rt, "DT"] + evaluate_result(y_pred_DT, y_test))
        csv_writer.writerow(["RP", rt, "RF"] + evaluate_result(y_pred_RF, y_test))
        csv_writer.writerow(["RP", rt, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
        csv_writer.writerow(["RP", rt, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
        csv_writer.writerow(["RP", rt, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))
    
    ########## Diveplane 1 run ##########
    print("----- diveplane 1 -----")
    tar = y_train.name
    conditions = [{tar: 1},
                  {tar: 0}] * (int(X_train.shape[0] / 2))
    
    X_train_diveplane_1 = X_train.copy()
    y_train_diveplane_1 = y_train.copy()
    X_train_diveplane_1[tar] = y_train_diveplane_1
    
    features = infer_feature_attributes(X_train_diveplane_1)
    for f_name, f_value in features.items():
        if f_value["type"] == "nominal":
            f_value["non_sensitive"] = True

    start_time = time.time()
    g = Geminai()
    g.train(X_train_diveplane_1, features=features)

    gen_df = g.synthesize_cases(
        n_samples=len(conditions),
        case_context_values_maps=conditions,
        desired_conviction=5,
        generate_new_cases="no"
    )

    rt = time.time() - start_time

    X_train_new = gen_df.iloc[:, :-1]
    y_train_new = gen_df.iloc[:, -1]
    
    # scale data
    scaler = StandardScaler()
    X_train_scale = pd.DataFrame(scaler.fit_transform(X_train_new), columns=X_train_new.columns, index=X_train_new.index)
    X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

    print("y train ratio: 1:" + str(round(y_train_new.value_counts()[0] / y_train_new.value_counts()[1])))

    # create models
    clf_SVM = SVC()
    clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
    clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
    clf_DT = DecisionTreeClassifier()
    clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
    clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
    clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
    clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

    clf_SVM.fit(X_train_scale, y_train_new)
    clf_KNN.fit(X_train_scale, y_train_new)
    clf_LR.fit(X_train_scale, y_train_new)
    clf_DT.fit(X_train_scale, y_train_new)
    clf_RF.fit(X_train_scale, y_train_new)
    clf_LightGBM.fit(X_train_scale, y_train_new)
    clf_Adaboost.fit(X_train_scale, y_train_new)
    clf_GBDT.fit(X_train_scale, y_train_new)

    y_pred_SVM = clf_SVM.predict(X_test_scale)
    y_pred_KNN = clf_KNN.predict(X_test_scale)
    y_pred_LR = clf_LR.predict(X_test_scale)
    y_pred_DT = clf_DT.predict(X_test_scale)
    y_pred_RF = clf_RF.predict(X_test_scale)
    y_pred_LightGBM = clf_LightGBM.predict(X_test_scale)
    y_pred_Adaboost = clf_Adaboost.predict(X_test_scale)
    y_pred_GBDT = clf_GBDT.predict(X_test_scale)
    
    with open(write_path, "a", newline="") as f:
        csv_writer = csv.writer(f)

        csv_writer.writerow(["Diveplane1", rt, "SVM"] + evaluate_result(y_pred_SVM, y_test))
        csv_writer.writerow(["Diveplane1", rt, "KNN"] + evaluate_result(y_pred_KNN, y_test))
        csv_writer.writerow(["Diveplane1", rt, "LR"] + evaluate_result(y_pred_LR, y_test))
        csv_writer.writerow(["Diveplane1", rt, "DT"] + evaluate_result(y_pred_DT, y_test))
        csv_writer.writerow(["Diveplane1", rt, "RF"] + evaluate_result(y_pred_RF, y_test))
        csv_writer.writerow(["Diveplane1", rt, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
        csv_writer.writerow(["Diveplane1", rt, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
        csv_writer.writerow(["Diveplane1", rt, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))
    
    ########## Diveplane 2 run ##########
    print("----- diveplane 2 -----")
    tar = y_train.name
    n_cases_diff = y_train.value_counts()[0] - y_train.value_counts()[1]
    conditions = [{tar: 1}] * n_cases_diff
    
    X_train_diveplane_2 = X_train.copy()
    y_train_diveplane_2 = y_train.copy()
    X_train_diveplane_2[tar] = y_train_diveplane_2
    
    features = infer_feature_attributes(X_train_diveplane_2)
    for f_name, f_value in features.items():
        if f_value["type"] == "nominal":
            f_value["non_sensitive"] = True

    start_time = time.time()
    g = Geminai()
    g.train(X_train_diveplane_2, features=features)

    gen_df = g.synthesize_cases(
        n_samples=len(conditions),
        case_context_values_maps=conditions,
        desired_conviction=5,
        generate_new_cases="no"
    )

    rt = time.time() - start_time

    X_train_new = pd.concat([X_train_diveplane_2, gen_df], ignore_index=True, axis=0)
    y_train_new = X_train_new.iloc[:, -1]
    X_train_new = X_train_new.iloc[:, :-1]
    
    # scale data
    scaler = StandardScaler()
    X_train_scale = pd.DataFrame(scaler.fit_transform(X_train_new), columns=X_train_new.columns, index=X_train_new.index)
    X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)
    
    print("y train ratio: 1:" + str(round(y_train_new.value_counts()[0] / y_train_new.value_counts()[1])))

    # create models
    clf_SVM = SVC()
    clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
    clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
    clf_DT = DecisionTreeClassifier()
    clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
    clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
    clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
    clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

    clf_SVM.fit(X_train_scale, y_train_new)
    clf_KNN.fit(X_train_scale, y_train_new)
    clf_LR.fit(X_train_scale, y_train_new)
    clf_DT.fit(X_train_scale, y_train_new)
    clf_RF.fit(X_train_scale, y_train_new)
    clf_LightGBM.fit(X_train_scale, y_train_new)
    clf_Adaboost.fit(X_train_scale, y_train_new)
    clf_GBDT.fit(X_train_scale, y_train_new)

    y_pred_SVM = clf_SVM.predict(X_test_scale)
    y_pred_KNN = clf_KNN.predict(X_test_scale)
    y_pred_LR = clf_LR.predict(X_test_scale)
    y_pred_DT = clf_DT.predict(X_test_scale)
    y_pred_RF = clf_RF.predict(X_test_scale)
    y_pred_LightGBM = clf_LightGBM.predict(X_test_scale)
    y_pred_Adaboost = clf_Adaboost.predict(X_test_scale)
    y_pred_GBDT = clf_GBDT.predict(X_test_scale)
    
    with open(write_path, "a", newline="") as f:
        csv_writer = csv.writer(f)

        csv_writer.writerow(["Diveplane2", rt, "SVM"] + evaluate_result(y_pred_SVM, y_test))
        csv_writer.writerow(["Diveplane2", rt, "KNN"] + evaluate_result(y_pred_KNN, y_test))
        csv_writer.writerow(["Diveplane2", rt, "LR"] + evaluate_result(y_pred_LR, y_test))
        csv_writer.writerow(["Diveplane2", rt, "DT"] + evaluate_result(y_pred_DT, y_test))
        csv_writer.writerow(["Diveplane2", rt, "RF"] + evaluate_result(y_pred_RF, y_test))
        csv_writer.writerow(["Diveplane2", rt, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
        csv_writer.writerow(["Diveplane2", rt, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
        csv_writer.writerow(["Diveplane2", rt, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))
    
    ########## DS run ##########
    print("----- DS -----")
    mode = "independent_attribute_mode"

    col = X_train.columns
    tar = y_train.name
    
    X_train_DS = X_train.copy()
    y_train_DS = y_train.copy()
    X_train_DS[tar] = y_train_DS
    write_df = X_train_DS[X_train_DS[tar] == 1]
    write_df = write_df.iloc[:, :-1]
    write_df.to_csv(f"{os.path.dirname(os.getcwd())}/extra/js_vuln_pos_df.csv", index=False)
    
    threshold = 20
    num_tuples_to_generate = int(y_train.value_counts()[0] - y_train.value_counts()[1])

    start_time = time.time()

    description_file = f"{os.path.dirname(os.getcwd())}/extra/js_vuln.json"
    describer = DataDescriber(category_threshold=threshold)
    describer.describe_dataset_in_independent_attribute_mode(
        dataset_file=f"{os.path.dirname(os.getcwd())}/extra/js_vuln_pos_df.csv"
    )
    describer.save_dataset_description_to_file(description_file)

    generator = DataGenerator()
    generator.generate_dataset_in_independent_mode(num_tuples_to_generate, description_file)
    generator.save_synthetic_data(f"{os.path.dirname(os.getcwd())}/extra/js_vuln_syn_df.csv")

    rt = time.time() - start_time

    X_train_new = pd.read_csv(f"{os.path.dirname(os.getcwd())}/extra/js_vuln_syn_df.csv").to_numpy()
    y_train_new = np.ones(num_tuples_to_generate)
    X_train_new = pd.DataFrame(np.vstack((X_train.to_numpy(), X_train_new)), columns=col)
    y_train_new = pd.Series(np.hstack((y_train.to_numpy(), y_train_new)), name=tar)
    
    # scale data
    scaler = StandardScaler()
    X_train_scale = pd.DataFrame(scaler.fit_transform(X_train_new), columns=X_train_new.columns, index=X_train_new.index)
    X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

    print("y train ratio: 1:" + str(round(y_train_new.value_counts()[0] / y_train_new.value_counts()[1])))

    # create models
    clf_SVM = SVC()
    clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
    clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
    clf_DT = DecisionTreeClassifier()
    clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
    clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
    clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
    clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

    clf_SVM.fit(X_train_scale, y_train_new)
    clf_KNN.fit(X_train_scale, y_train_new)
    clf_LR.fit(X_train_scale, y_train_new)
    clf_DT.fit(X_train_scale, y_train_new)
    clf_RF.fit(X_train_scale, y_train_new)
    clf_LightGBM.fit(X_train_scale, y_train_new)
    clf_Adaboost.fit(X_train_scale, y_train_new)
    clf_GBDT.fit(X_train_scale, y_train_new)

    y_pred_SVM = clf_SVM.predict(X_test_scale)
    y_pred_KNN = clf_KNN.predict(X_test_scale)
    y_pred_LR = clf_LR.predict(X_test_scale)
    y_pred_DT = clf_DT.predict(X_test_scale)
    y_pred_RF = clf_RF.predict(X_test_scale)
    y_pred_LightGBM = clf_LightGBM.predict(X_test_scale)
    y_pred_Adaboost = clf_Adaboost.predict(X_test_scale)
    y_pred_GBDT = clf_GBDT.predict(X_test_scale)
    
    with open(write_path, "a", newline="") as f:
        csv_writer = csv.writer(f)

        csv_writer.writerow(["DS", rt, "SVM"] + evaluate_result(y_pred_SVM, y_test))
        csv_writer.writerow(["DS", rt, "KNN"] + evaluate_result(y_pred_KNN, y_test))
        csv_writer.writerow(["DS", rt, "LR"] + evaluate_result(y_pred_LR, y_test))
        csv_writer.writerow(["DS", rt, "DT"] + evaluate_result(y_pred_DT, y_test))
        csv_writer.writerow(["DS", rt, "RF"] + evaluate_result(y_pred_RF, y_test))
        csv_writer.writerow(["DS", rt, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
        csv_writer.writerow(["DS", rt, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
        csv_writer.writerow(["DS", rt, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))
    
    ########## SDV FASTML run ##########
    print("----- SDV FASTML -----")
    col = X_train.columns
    tar = y_train.name
    num_tuples_to_generate = int(y_train.value_counts()[0] - y_train.value_counts()[1])
    
    X_train_SDV = X_train.copy()
    y_train_SDV = y_train.copy()
    X_train_SDV[tar] = y_train_SDV
    pos_df = X_train_SDV[X_train_SDV[tar] == 1]
    pos_df = pos_df.iloc[:, :-1]

    metadata = SingleTableMetadata()
    metadata.detect_from_dataframe(data=pos_df)
    
    start_time = time.time()
    syn1 = SingleTablePreset(metadata, name="FAST_ML")
    syn1.fit(data=pos_df)
    X_train_new = syn1.sample(num_rows=num_tuples_to_generate).to_numpy()

    rt = time.time() - start_time

    X_train_new = pd.DataFrame(np.vstack((X_train.to_numpy(), X_train_new)), columns=col)
    y_train_new = np.ones(num_tuples_to_generate)
    y_train_new = pd.Series(np.hstack((y_train.to_numpy(), y_train_new)), name=tar)

    # scale data
    scaler = StandardScaler()
    X_train_scale = pd.DataFrame(scaler.fit_transform(X_train_new), columns=X_train_new.columns, index=X_train_new.index)
    X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

    print("y train ratio: 1:" + str(round(y_train_new.value_counts()[0] / y_train_new.value_counts()[1])))

    # create models
    clf_SVM = SVC()
    clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
    clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
    clf_DT = DecisionTreeClassifier()
    clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
    clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
    clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
    clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

    clf_SVM.fit(X_train_scale, y_train_new)
    clf_KNN.fit(X_train_scale, y_train_new)
    clf_LR.fit(X_train_scale, y_train_new)
    clf_DT.fit(X_train_scale, y_train_new)
    clf_RF.fit(X_train_scale, y_train_new)
    clf_LightGBM.fit(X_train_scale, y_train_new)
    clf_Adaboost.fit(X_train_scale, y_train_new)
    clf_GBDT.fit(X_train_scale, y_train_new)

    y_pred_SVM = clf_SVM.predict(X_test_scale)
    y_pred_KNN = clf_KNN.predict(X_test_scale)
    y_pred_LR = clf_LR.predict(X_test_scale)
    y_pred_DT = clf_DT.predict(X_test_scale)
    y_pred_RF = clf_RF.predict(X_test_scale)
    y_pred_LightGBM = clf_LightGBM.predict(X_test_scale)
    y_pred_Adaboost = clf_Adaboost.predict(X_test_scale)
    y_pred_GBDT = clf_GBDT.predict(X_test_scale)
    
    with open(write_path, "a", newline="") as f:
        csv_writer = csv.writer(f)

        csv_writer.writerow(["SDV_FASTML", rt, "SVM"] + evaluate_result(y_pred_SVM, y_test))
        csv_writer.writerow(["SDV_FASTML", rt, "KNN"] + evaluate_result(y_pred_KNN, y_test))
        csv_writer.writerow(["SDV_FASTML", rt, "LR"] + evaluate_result(y_pred_LR, y_test))
        csv_writer.writerow(["SDV_FASTML", rt, "DT"] + evaluate_result(y_pred_DT, y_test))
        csv_writer.writerow(["SDV_FASTML", rt, "RF"] + evaluate_result(y_pred_RF, y_test))
        csv_writer.writerow(["SDV_FASTML", rt, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
        csv_writer.writerow(["SDV_FASTML", rt, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
        csv_writer.writerow(["SDV_FASTML", rt, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))
    
    ########## SDV GC ##########
    print("----- SDV GC -----")
    start_time = time.time()
    syn2 = GaussianCopulaSynthesizer(metadata)
    syn2.fit(data=pos_df)
    X_train_new = syn2.sample(num_rows=num_tuples_to_generate).to_numpy()

    rt = time.time() - start_time

    X_train_new = pd.DataFrame(np.vstack((X_train.to_numpy(), X_train_new)), columns=col)
    y_train_new = np.ones(num_tuples_to_generate)
    y_train_new = pd.Series(np.hstack((y_train.to_numpy(), y_train_new)), name=tar)

    # scale data
    scaler = StandardScaler()
    X_train_scale = pd.DataFrame(scaler.fit_transform(X_train_new), columns=X_train_new.columns, index=X_train_new.index)
    X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

    print("y train ratio: 1:" + str(round(y_train_new.value_counts()[0] / y_train_new.value_counts()[1])))

    # create models
    clf_SVM = SVC()
    clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
    clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
    clf_DT = DecisionTreeClassifier()
    clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
    clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
    clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
    clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

    clf_SVM.fit(X_train_scale, y_train_new)
    clf_KNN.fit(X_train_scale, y_train_new)
    clf_LR.fit(X_train_scale, y_train_new)
    clf_DT.fit(X_train_scale, y_train_new)
    clf_RF.fit(X_train_scale, y_train_new)
    clf_LightGBM.fit(X_train_scale, y_train_new)
    clf_Adaboost.fit(X_train_scale, y_train_new)
    clf_GBDT.fit(X_train_scale, y_train_new)

    y_pred_SVM = clf_SVM.predict(X_test_scale)
    y_pred_KNN = clf_KNN.predict(X_test_scale)
    y_pred_LR = clf_LR.predict(X_test_scale)
    y_pred_DT = clf_DT.predict(X_test_scale)
    y_pred_RF = clf_RF.predict(X_test_scale)
    y_pred_LightGBM = clf_LightGBM.predict(X_test_scale)
    y_pred_Adaboost = clf_Adaboost.predict(X_test_scale)
    y_pred_GBDT = clf_GBDT.predict(X_test_scale)
    
    with open(write_path, "a", newline="") as f:
        csv_writer = csv.writer(f)

        csv_writer.writerow(["SDV_GC", rt, "SVM"] + evaluate_result(y_pred_SVM, y_test))
        csv_writer.writerow(["SDV_GC", rt, "KNN"] + evaluate_result(y_pred_KNN, y_test))
        csv_writer.writerow(["SDV_GC", rt, "LR"] + evaluate_result(y_pred_LR, y_test))
        csv_writer.writerow(["SDV_GC", rt, "DT"] + evaluate_result(y_pred_DT, y_test))
        csv_writer.writerow(["SDV_GC", rt, "RF"] + evaluate_result(y_pred_RF, y_test))
        csv_writer.writerow(["SDV_GC", rt, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
        csv_writer.writerow(["SDV_GC", rt, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
        csv_writer.writerow(["SDV_GC", rt, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))
    
    ########## SDV GAN ##########
    print("----- SDV GAN -----")
    start_time = time.time()
    syn3 = CTGANSynthesizer(metadata)
    syn3.fit(data=pos_df)
    X_train_new = syn3.sample(num_rows=num_tuples_to_generate).to_numpy()

    rt = time.time() - start_time

    X_train_new = pd.DataFrame(np.vstack((X_train.to_numpy(), X_train_new)), columns=col)
    y_train_new = np.ones(num_tuples_to_generate)
    y_train_new = pd.Series(np.hstack((y_train.to_numpy(), y_train_new)), name=tar)

    # scale data
    scaler = StandardScaler()
    X_train_scale = pd.DataFrame(scaler.fit_transform(X_train_new), columns=X_train_new.columns, index=X_train_new.index)
    X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

    print("y train ratio: 1:" + str(round(y_train_new.value_counts()[0] / y_train_new.value_counts()[1])))

    # create models
    clf_SVM = SVC()
    clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
    clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
    clf_DT = DecisionTreeClassifier()
    clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
    clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
    clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
    clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

    clf_SVM.fit(X_train_scale, y_train_new)
    clf_KNN.fit(X_train_scale, y_train_new)
    clf_LR.fit(X_train_scale, y_train_new)
    clf_DT.fit(X_train_scale, y_train_new)
    clf_RF.fit(X_train_scale, y_train_new)
    clf_LightGBM.fit(X_train_scale, y_train_new)
    clf_Adaboost.fit(X_train_scale, y_train_new)
    clf_GBDT.fit(X_train_scale, y_train_new)

    y_pred_SVM = clf_SVM.predict(X_test_scale)
    y_pred_KNN = clf_KNN.predict(X_test_scale)
    y_pred_LR = clf_LR.predict(X_test_scale)
    y_pred_DT = clf_DT.predict(X_test_scale)
    y_pred_RF = clf_RF.predict(X_test_scale)
    y_pred_LightGBM = clf_LightGBM.predict(X_test_scale)
    y_pred_Adaboost = clf_Adaboost.predict(X_test_scale)
    y_pred_GBDT = clf_GBDT.predict(X_test_scale)
    
    with open(write_path, "a", newline="") as f:
        csv_writer = csv.writer(f)

        csv_writer.writerow(["SDV_GAN", rt, "SVM"] + evaluate_result(y_pred_SVM, y_test))
        csv_writer.writerow(["SDV_GAN", rt, "KNN"] + evaluate_result(y_pred_KNN, y_test))
        csv_writer.writerow(["SDV_GAN", rt, "LR"] + evaluate_result(y_pred_LR, y_test))
        csv_writer.writerow(["SDV_GAN", rt, "DT"] + evaluate_result(y_pred_DT, y_test))
        csv_writer.writerow(["SDV_GAN", rt, "RF"] + evaluate_result(y_pred_RF, y_test))
        csv_writer.writerow(["SDV_GAN", rt, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
        csv_writer.writerow(["SDV_GAN", rt, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
        csv_writer.writerow(["SDV_GAN", rt, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))

----- in repeat 9 -----
y value counts: 
 0    10629
1     1496
Name: Vuln, dtype: int64
y class ratio: 1: 7
--- y train classes count: 
0    8503
1    1197
Name: Vuln, dtype: int64
--- y train ratio: 1:7
 
--- y test classes count: 
0    2126
1     299
Name: Vuln, dtype: int64
--- y test ratio: 1:7
----- normal -----
----- random -----
y train ratio: 1:1
----- ADASYN ------
y train ratio: 1:1
----- borderlineSMOTE -----
y train ratio: 1:1
----- SMOTE -----
y train ratio: 1:1
----- SVMSMOTE -----
y train ratio: 1:1
----- SMOTUNED -----
y train ratio of SVM: 1:1
y train ratio of KNN: 1:1
y train ratio of LR: 1:1
y train ratio of DT: 1:1
y train ratio of RF: 1:1
y train ratio of LightGBM: 1:1
y train ratio of Adaboost: 1:1
y train ratio of GBDT: 1:1
----- DAZZLE -----
  0%|                                                                           | 0/100 [00:00<?, ?trial/s, best loss=?]

2023-09-25 12:39:26.080665: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-25 12:39:26.080974: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


100%|██████████████████████████████████████████████| 100/100 [12:30<00:00,  7.50s/trial, best loss: -0.8982353609550662]
Best Hyperparameters: {'batch_size': 128, 'discriminator_activation_fn': <function sigmoid at 0x7f7b4bc333a0>, 'discriminator_layer_normalization': False, 'discriminator_lr': 0.06007113581560754, 'discriminator_optimizer': <class 'keras.src.optimizers.adadelta.Adadelta'>, 'epochs': 146, 'generator_activation_fn': <function sigmoid at 0x7f7b4bc333a0>, 'generator_layer_normalization': True, 'generator_lr': 0.010440528944188773, 'generator_optimizer': <class 'keras.src.optimizers.adagrad.Adagrad'>}
Best G-Measure: 0.8982353609550662
 27/304 [=>............................] - ETA: 1s

/home/lingxiao/diveplane/lib/python3.8/site-packages/tensorflow/python/data/ops/structured_function.py:265: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(
/home/lingxiao/diveplane/lib/python3.8/site-packages/tensorflow/python/data/ops/structured_function.py:265: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


304/304 [==============================] - 1s 4ms/step
y train ratio: 1:1
----- WGAN -----
MAX CLASS 8503
CLASS ID 1
Epoch 1/150 completed. Gen loss: 0.03366551548242569. Desc loss_real: -0.007228841073811054. Desc loss_fake: -0.03366551548242569
Epoch 51/150 completed. Gen loss: 0.00011562809231691062. Desc loss_real: 0.0020602773874998093. Desc loss_fake: -0.00011562809231691062
Epoch 101/150 completed. Gen loss: 0.0001563505211379379. Desc loss_real: 0.002417274285107851. Desc loss_fake: -0.0001563505211379379


...

/home/lingxiao/diveplane/lib/python3.8/site-packages/rdt/transformers/numerical.py:112: UserWarning: No rounding scheme detected for column 'HDIFF'. Data will not be rounded.
  warnings.warn(
/home/lingxiao/diveplane/lib/python3.8/site-packages/rdt/transformers/numerical.py:112: UserWarning: No rounding scheme detected for column 'HVOL'. Data will not be rounded.
  warnings.warn(
/home/lingxiao/diveplane/lib/python3.8/site-packages/rdt/transformers/numerical.py:112: UserWarning: No rounding scheme detected for column 'HEFF'. Data will not be rounded.
  warnings.warn(
/home/lingxiao/diveplane/lib/python3.8/site-packages/rdt/transformers/numerical.py:112: UserWarning: No rounding scheme detected for column 'HBUGS'. Data will not be rounded.
  warnings.warn(
/home/lingxiao/diveplane/lib/python3.8/site-packages/rdt/transformers/numerical.py:112: UserWarning: No rounding scheme detected for column 'HTIME'. Data will not be rounded.
  warnings.warn(
/home/lingxiao/diveplane/lib/python3.8/sit

y train ratio: 1:1
----- SDV GAN -----


/home/lingxiao/diveplane/lib/python3.8/site-packages/rdt/transformers/numerical.py:112: UserWarning: No rounding scheme detected for column 'HDIFF'. Data will not be rounded.
  warnings.warn(
/home/lingxiao/diveplane/lib/python3.8/site-packages/rdt/transformers/numerical.py:112: UserWarning: No rounding scheme detected for column 'HVOL'. Data will not be rounded.
  warnings.warn(
/home/lingxiao/diveplane/lib/python3.8/site-packages/rdt/transformers/numerical.py:112: UserWarning: No rounding scheme detected for column 'HEFF'. Data will not be rounded.
  warnings.warn(
/home/lingxiao/diveplane/lib/python3.8/site-packages/rdt/transformers/numerical.py:112: UserWarning: No rounding scheme detected for column 'HBUGS'. Data will not be rounded.
  warnings.warn(
/home/lingxiao/diveplane/lib/python3.8/site-packages/rdt/transformers/numerical.py:112: UserWarning: No rounding scheme detected for column 'HTIME'. Data will not be rounded.
  warnings.warn(
/home/lingxiao/diveplane/lib/python3.8/sit

/home/lingxiao/diveplane/lib/python3.8/site-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
/home/lingxiao/diveplane/lib/python3.8/site-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
/home/lingxiao/diveplane/lib/python3.8/site-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
/home/lingxiao/diveplane/lib/python3.8/site-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missi

y train ratio: 1:1


In [5]:
rnl = [256, 116, 328, 426, 92, 281, 439, 153, 64, 70]

for idx, rn in enumerate(rnl):
    write_file = f"JS_Vuln_nodrop_res_r{idx+1}_rn{rn}.csv"
    write_path = f"{os.path.dirname(os.getcwd())}/result/JS_Vuln_nodrop/{write_file}"
    data_path = f"{os.path.dirname(os.getcwd())}/data/JavaScript_Vulnerability/"
    datafiles = [f for f in os.listdir(data_path) if f.endswith("csv")]

    df = pd.read_csv(f"{data_path}/{datafiles[0]}")
    drop_columns = ["name", "longname", "path", "full_repo_path", "line", "column", "endline", "endcolumn"]
    df = df.drop(drop_columns, axis=1)
    # df = df.drop_duplicates()
    # df.reset_index(inplace=True, drop=True)

    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]
    print("y value counts: \n", str(y.value_counts()))
    print("y class ratio: 1:", str(round(y.value_counts()[0]/y.value_counts()[1])))

    cols = X.columns
    tar = y.name

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=rn)
    print("--- y train classes count: \n" + str(y_train.value_counts()))
    print("--- y train ratio: 1:" + str(round(y_train.value_counts()[0] / y_train.value_counts()[1])))
    print(" ")
    print("--- y test classes count: \n" + str(y_test.value_counts()))
    print("--- y test ratio: 1:" + str(round(y_test.value_counts()[0] / y_test.value_counts()[1])))

    ########## DAZZLE run ##########
    # DAZZLE oversampling run - DAZZLE oversampling technique
    print("----- DAZZLE1 -----")

    X_train_GAN = X_train.copy()
    y_train_GAN = y_train.copy()
    X_test_GAN = X_test.copy()
    y_test_GAN = y_test.copy()
    rt, X_train_new, y_train_new, X_test_scale = DAZZLE1Oversampling(X_train=X_train_GAN, 
                                                                     y_train=y_train_GAN, 
                                                                     X_test=X_test_GAN,
                                                                     y_test=y_test_GAN)

    X_train_new = pd.DataFrame(X_train_new, columns=cols)
    y_train_new = pd.Series(y_train_new, name=tar)
    X_test_scale = pd.DataFrame(X_test_scale, columns=cols)
    # scale data
    # scaler = StandardScaler()
    # X_train_scale = pd.DataFrame(scaler.fit_transform(X_train_new), columns=X_train_new.columns, index=X_train_new.index)
    # X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

    print("y train ratio: 1:" + str(round(y_train_new.value_counts()[0] / y_train_new.value_counts()[1])))

    # create models
    clf_SVM = SVC()
    clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
    clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
    clf_DT = DecisionTreeClassifier()
    clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
    clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
    clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
    clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

    clf_SVM.fit(X_train_new, y_train_new)
    clf_KNN.fit(X_train_new, y_train_new)
    clf_LR.fit(X_train_new, y_train_new)
    clf_DT.fit(X_train_new, y_train_new)
    clf_RF.fit(X_train_new, y_train_new)
    clf_LightGBM.fit(X_train_new, y_train_new)
    clf_Adaboost.fit(X_train_new, y_train_new)
    clf_GBDT.fit(X_train_new, y_train_new)

    y_pred_SVM = clf_SVM.predict(X_test_scale)
    y_pred_KNN = clf_KNN.predict(X_test_scale)
    y_pred_LR = clf_LR.predict(X_test_scale)
    y_pred_DT = clf_DT.predict(X_test_scale)
    y_pred_RF = clf_RF.predict(X_test_scale)
    y_pred_LightGBM = clf_LightGBM.predict(X_test_scale)
    y_pred_Adaboost = clf_Adaboost.predict(X_test_scale)
    y_pred_GBDT = clf_GBDT.predict(X_test_scale)

    with open(write_path, "a", newline="") as f:
        csv_writer = csv.writer(f)

        csv_writer.writerow(["DAZZLE1", rt, "SVM"] + evaluate_result(y_pred_SVM, y_test))
        csv_writer.writerow(["DAZZLE1", rt, "KNN"] + evaluate_result(y_pred_KNN, y_test))
        csv_writer.writerow(["DAZZLE1", rt, "LR"] + evaluate_result(y_pred_LR, y_test))
        csv_writer.writerow(["DAZZLE1", rt, "DT"] + evaluate_result(y_pred_DT, y_test))
        csv_writer.writerow(["DAZZLE1", rt, "RF"] + evaluate_result(y_pred_RF, y_test))
        csv_writer.writerow(["DAZZLE1", rt, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
        csv_writer.writerow(["DAZZLE1", rt, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
        csv_writer.writerow(["DAZZLE1", rt, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))

y value counts: 
 0    10629
1     1496
Name: Vuln, dtype: int64
y class ratio: 1: 7
--- y train classes count: 
0    8503
1    1197
Name: Vuln, dtype: int64
--- y train ratio: 1:7
 
--- y test classes count: 
0    2126
1     299
Name: Vuln, dtype: int64
--- y test ratio: 1:7
----- DAZZLE1 -----
[    0/46664] LG:-0.062 LD:-1.137 D:0.077 GP:0.121 RMSEAVG:0.153 NUM:0.153 SynTraiAuc:0.601 RFAcc:1.000                 
[  500/46664] LG:0.496 LD:0.350 D:0.385 GP:0.003 RMSEAVG:0.153 NUM:0.153 SynTraiAuc:0.601 RFAcc:1.000                   
[ 1000/46664] LG:0.659 LD:0.236 D:0.297 GP:0.006 RMSEAVG:0.153 NUM:0.153 SynTraiAuc:0.601 RFAcc:1.000                   
[ 1500/46664] LG:0.759 LD:0.199 D:0.243 GP:0.004 RMSEAVG:0.028 NUM:0.028 SynTraiAuc:0.601 RFAcc:0.990                   
[ 2000/46664] LG:0.842 LD:0.347 D:0.371 GP:0.002 RMSEAVG:0.028 NUM:0.028 SynTraiAuc:0.601 RFAcc:0.990                   
[ 2500/46664] LG:0.935 LD:0.242 D:0.274 GP:0.003 RMSEAVG:0.029 NUM:0.029 SynTraiAuc:0.601 RFAcc:0.

[32500/46664] LG:0.708 LD:0.054 D:0.069 GP:0.001 RMSEAVG:0.013 NUM:0.013 SynTraiAuc:0.716 RFAcc:0.985                   
[33000/46664] LG:0.708 LD:0.022 D:0.050 GP:0.003 RMSEAVG:0.013 NUM:0.013 SynTraiAuc:0.716 RFAcc:0.985                   
[33500/46664] LG:0.769 LD:0.133 D:0.154 GP:0.002 RMSEAVG:0.013 NUM:0.013 SynTraiAuc:0.716 RFAcc:0.985                   
[34000/46664] LG:0.712 LD:0.257 D:0.285 GP:0.003 RMSEAVG:0.023 NUM:0.023 SynTraiAuc:0.716 RFAcc:0.981                   
[34500/46664] LG:0.705 LD:0.046 D:0.083 GP:0.004 RMSEAVG:0.023 NUM:0.023 SynTraiAuc:0.716 RFAcc:0.981                   
[35000/46664] LG:0.734 LD:0.203 D:0.229 GP:0.003 RMSEAVG:0.024 NUM:0.024 SynTraiAuc:0.714 RFAcc:0.985                   
[35500/46664] LG:0.635 LD:0.122 D:0.144 GP:0.002 RMSEAVG:0.024 NUM:0.024 SynTraiAuc:0.714 RFAcc:0.985                   
[36000/46664] LG:0.598 LD:0.104 D:0.137 GP:0.003 RMSEAVG:0.024 NUM:0.024 SynTraiAuc:0.714 RFAcc:0.985                   
[36500/46664] LG:0.595 LD:0.131 

[17500/143252] LG:1.032 LD:0.245 D:0.281 GP:0.002 RMSEAVG:0.045 NUM:0.045 SynTraiAuc:0.647 RFAcc:1.000                  
[18000/143252] LG:0.954 LD:0.335 D:0.384 GP:0.003 RMSEAVG:0.045 NUM:0.045 SynTraiAuc:0.647 RFAcc:1.000                  
[18500/143252] LG:1.003 LD:0.283 D:0.314 GP:0.002 RMSEAVG:0.045 NUM:0.045 SynTraiAuc:0.647 RFAcc:1.000                  
[19000/143252] LG:0.967 LD:0.246 D:0.273 GP:0.001 RMSEAVG:0.045 NUM:0.045 SynTraiAuc:0.647 RFAcc:0.995                  
[19500/143252] LG:1.029 LD:0.151 D:0.266 GP:0.006 RMSEAVG:0.045 NUM:0.045 SynTraiAuc:0.647 RFAcc:0.995                  
[20000/143252] LG:1.004 LD:0.346 D:0.365 GP:0.001 RMSEAVG:0.031 NUM:0.031 SynTraiAuc:0.685 RFAcc:0.975                  
[20500/143252] LG:1.003 LD:0.345 D:0.385 GP:0.002 RMSEAVG:0.031 NUM:0.031 SynTraiAuc:0.685 RFAcc:0.975                  
[21000/143252] LG:1.056 LD:0.279 D:0.314 GP:0.002 RMSEAVG:0.031 NUM:0.031 SynTraiAuc:0.685 RFAcc:0.975                  
[21500/143252] LG:1.051 LD:0.244

[51000/143252] LG:1.148 LD:0.238 D:0.244 GP:0.000 RMSEAVG:0.040 NUM:0.040 SynTraiAuc:0.675 RFAcc:0.979                  
[51500/143252] LG:1.169 LD:0.197 D:0.240 GP:0.002 RMSEAVG:0.058 NUM:0.058 SynTraiAuc:0.675 RFAcc:1.000                  
[52000/143252] LG:1.156 LD:0.231 D:0.266 GP:0.002 RMSEAVG:0.058 NUM:0.058 SynTraiAuc:0.675 RFAcc:1.000                  
[52500/143252] LG:1.039 LD:0.084 D:0.101 GP:0.001 RMSEAVG:0.044 NUM:0.044 SynTraiAuc:0.675 RFAcc:1.000                  
[53000/143252] LG:1.104 LD:0.050 D:0.063 GP:0.001 RMSEAVG:0.044 NUM:0.044 SynTraiAuc:0.675 RFAcc:1.000                  
[53500/143252] LG:1.161 LD:0.248 D:0.271 GP:0.001 RMSEAVG:0.044 NUM:0.044 SynTraiAuc:0.675 RFAcc:1.000                  
[54000/143252] LG:1.112 LD:0.264 D:0.313 GP:0.003 RMSEAVG:0.057 NUM:0.057 SynTraiAuc:0.675 RFAcc:0.995                  
[54500/143252] LG:1.113 LD:0.420 D:0.477 GP:0.003 RMSEAVG:0.057 NUM:0.057 SynTraiAuc:0.675 RFAcc:0.995                  
[55000/143252] LG:1.091 LD:0.009

[84500/143252] LG:0.973 LD:0.100 D:0.152 GP:0.003 RMSEAVG:0.036 NUM:0.036 SynTraiAuc:0.576 RFAcc:1.000                  
[85000/143252] LG:0.899 LD:0.094 D:0.136 GP:0.002 RMSEAVG:0.048 NUM:0.048 SynTraiAuc:0.619 RFAcc:1.000                  
[85500/143252] LG:0.869 LD:-0.029 D:-0.007 GP:0.001 RMSEAVG:0.048 NUM:0.048 SynTraiAuc:0.619 RFAcc:1.000                
[86000/143252] LG:0.948 LD:0.135 D:0.168 GP:0.002 RMSEAVG:0.048 NUM:0.048 SynTraiAuc:0.619 RFAcc:1.000                  
[86500/143252] LG:0.994 LD:-0.005 D:0.050 GP:0.003 RMSEAVG:0.029 NUM:0.029 SynTraiAuc:0.619 RFAcc:0.989                 
[87000/143252] LG:0.936 LD:0.250 D:0.339 GP:0.005 RMSEAVG:0.029 NUM:0.029 SynTraiAuc:0.619 RFAcc:0.989                  
[87500/143252] LG:0.946 LD:0.186 D:0.226 GP:0.002 RMSEAVG:0.055 NUM:0.055 SynTraiAuc:0.619 RFAcc:1.000                  
[88000/143252] LG:0.888 LD:-0.097 D:-0.073 GP:0.001 RMSEAVG:0.055 NUM:0.055 SynTraiAuc:0.619 RFAcc:1.000                
[88500/143252] LG:0.841 LD:0.102

...

[22500/158427] LG:0.839 LD:0.133 D:0.154 GP:0.001 RMSEAVG:0.045 NUM:0.045 SynTraiAuc:0.672 RFAcc:0.983                  
[23000/158427] LG:0.618 LD:0.150 D:0.169 GP:0.001 RMSEAVG:0.045 NUM:0.045 SynTraiAuc:0.672 RFAcc:0.983                  
[23500/158427] LG:0.817 LD:0.185 D:0.222 GP:0.002 RMSEAVG:0.045 NUM:0.045 SynTraiAuc:0.672 RFAcc:0.983                  
[24000/158427] LG:0.816 LD:0.120 D:0.173 GP:0.003 RMSEAVG:0.026 NUM:0.026 SynTraiAuc:0.672 RFAcc:0.979                  
[24500/158427] LG:0.656 LD:0.063 D:0.083 GP:0.001 RMSEAVG:0.026 NUM:0.026 SynTraiAuc:0.672 RFAcc:0.979                  
[25000/158427] LG:0.790 LD:0.168 D:0.185 GP:0.001 RMSEAVG:0.027 NUM:0.027 SynTraiAuc:0.606 RFAcc:0.959                  
[25500/158427] LG:0.737 LD:0.126 D:0.151 GP:0.002 RMSEAVG:0.027 NUM:0.027 SynTraiAuc:0.606 RFAcc:0.959                  
[26000/158427] LG:0.813 LD:0.120 D:0.134 GP:0.001 RMSEAVG:0.027 NUM:0.027 SynTraiAuc:0.606 RFAcc:0.959                  
[26500/158427] LG:0.917 LD:0.138

[56000/158427] LG:0.659 LD:0.073 D:0.082 GP:0.001 RMSEAVG:0.013 NUM:0.013 SynTraiAuc:0.594 RFAcc:0.949                  
[56500/158427] LG:0.655 LD:0.127 D:0.149 GP:0.001 RMSEAVG:0.016 NUM:0.016 SynTraiAuc:0.594 RFAcc:0.998                  
[57000/158427] LG:0.629 LD:0.070 D:0.252 GP:0.011 RMSEAVG:0.016 NUM:0.016 SynTraiAuc:0.594 RFAcc:0.998                  
[57500/158427] LG:0.625 LD:0.064 D:0.079 GP:0.001 RMSEAVG:0.032 NUM:0.032 SynTraiAuc:0.594 RFAcc:0.969                  
[58000/158427] LG:0.624 LD:0.110 D:0.122 GP:0.001 RMSEAVG:0.032 NUM:0.032 SynTraiAuc:0.594 RFAcc:0.969                  
[58500/158427] LG:0.610 LD:0.117 D:0.139 GP:0.001 RMSEAVG:0.032 NUM:0.032 SynTraiAuc:0.594 RFAcc:0.969                  
[59000/158427] LG:0.599 LD:0.056 D:0.073 GP:0.001 RMSEAVG:0.017 NUM:0.017 SynTraiAuc:0.594 RFAcc:0.980                  
[59500/158427] LG:0.617 LD:0.091 D:0.113 GP:0.001 RMSEAVG:0.017 NUM:0.017 SynTraiAuc:0.594 RFAcc:0.980                  
[60000/158427] LG:0.642 LD:0.115

[89500/158427] LG:0.396 LD:0.095 D:0.106 GP:0.001 RMSEAVG:0.026 NUM:0.026 SynTraiAuc:0.677 RFAcc:0.989                  
[90000/158427] LG:0.476 LD:0.242 D:0.250 GP:0.000 RMSEAVG:0.026 NUM:0.026 SynTraiAuc:0.668 RFAcc:0.979                  
[90500/158427] LG:0.376 LD:0.289 D:0.314 GP:0.002 RMSEAVG:0.026 NUM:0.026 SynTraiAuc:0.668 RFAcc:0.979                  
[91000/158427] LG:0.458 LD:0.105 D:0.117 GP:0.001 RMSEAVG:0.026 NUM:0.026 SynTraiAuc:0.668 RFAcc:0.979                  
[91500/158427] LG:0.457 LD:0.058 D:0.070 GP:0.001 RMSEAVG:0.026 NUM:0.026 SynTraiAuc:0.668 RFAcc:0.983                  
[92000/158427] LG:0.410 LD:0.185 D:0.208 GP:0.001 RMSEAVG:0.026 NUM:0.026 SynTraiAuc:0.668 RFAcc:0.983                  
[92500/158427] LG:0.394 LD:0.058 D:0.077 GP:0.001 RMSEAVG:0.021 NUM:0.021 SynTraiAuc:0.668 RFAcc:0.959                  
[93000/158427] LG:0.383 LD:0.083 D:0.091 GP:0.001 RMSEAVG:0.021 NUM:0.021 SynTraiAuc:0.668 RFAcc:0.959                  
[93500/158427] LG:0.393 LD:0.158

[123000/158427] LG:0.212 LD:0.105 D:0.134 GP:0.002 RMSEAVG:0.033 NUM:0.033 SynTraiAuc:0.602 RFAcc:0.980                 
[123500/158427] LG:0.273 LD:0.173 D:0.191 GP:0.001 RMSEAVG:0.033 NUM:0.033 SynTraiAuc:0.602 RFAcc:0.980                 
[124000/158427] LG:0.257 LD:0.040 D:0.057 GP:0.001 RMSEAVG:0.030 NUM:0.030 SynTraiAuc:0.602 RFAcc:0.981                 
[124500/158427] LG:0.256 LD:0.246 D:0.555 GP:0.019 RMSEAVG:0.030 NUM:0.030 SynTraiAuc:0.602 RFAcc:0.981                 
[125000/158427] LG:0.176 LD:0.073 D:0.092 GP:0.001 RMSEAVG:0.022 NUM:0.022 SynTraiAuc:0.730 RFAcc:0.999                 
[125500/158427] LG:0.202 LD:0.083 D:0.099 GP:0.001 RMSEAVG:0.022 NUM:0.022 SynTraiAuc:0.730 RFAcc:0.999                 
[126000/158427] LG:0.146 LD:0.089 D:0.133 GP:0.003 RMSEAVG:0.022 NUM:0.022 SynTraiAuc:0.730 RFAcc:0.999                 
[126500/158427] LG:0.282 LD:0.086 D:0.100 GP:0.001 RMSEAVG:0.024 NUM:0.024 SynTraiAuc:0.730 RFAcc:0.974                 
[127000/158427] LG:0.236 LD:0.15

[156500/158427] LG:0.072 LD:0.217 D:0.252 GP:0.002 RMSEAVG:0.014 NUM:0.014 SynTraiAuc:0.523 RFAcc:0.976                 
[157000/158427] LG:0.077 LD:0.139 D:0.157 GP:0.001 RMSEAVG:0.014 NUM:0.014 SynTraiAuc:0.523 RFAcc:0.976                 
[157500/158427] LG:0.010 LD:0.137 D:0.165 GP:0.002 RMSEAVG:0.035 NUM:0.035 SynTraiAuc:0.523 RFAcc:0.998                 
[158000/158427] LG:0.059 LD:0.090 D:0.197 GP:0.007 RMSEAVG:0.035 NUM:0.035 SynTraiAuc:0.523 RFAcc:0.998                 
[158427/158427] LG:0.043 LD:0.105 D:0.151 GP:0.003 RMSEAVG:0.035 NUM:0.035 SynTraiAuc:0.523 RFAcc:0.998                 
needed sample                                                                                                           
7306                                                                                                                    
100%|█████████████████████████████████████████████████| 5/5 [30:49<00:00, 369.97s/trial, best loss: 0.20637353702552408]
[ 5000/32832] LG:0.436 LD:0.120 

In [6]:
rnl = [35039, 25734, 217, 176, 372, 238, 480, 467, 310, 106]

for idx, rn in enumerate(rnl):
    write_file = f"JS_Vuln_res_r{idx+1}_rn{rn}.csv"
    write_path = f"{os.path.dirname(os.getcwd())}/result/JS_Vuln/{write_file}"
    data_path = f"{os.path.dirname(os.getcwd())}/data/JavaScript_Vulnerability/"
    datafiles = [f for f in os.listdir(data_path) if f.endswith("csv")]

    df = pd.read_csv(f"{data_path}/{datafiles[0]}")
    drop_columns = ["name", "longname", "path", "full_repo_path", "line", "column", "endline", "endcolumn"]
    df = df.drop(drop_columns, axis=1)
    df = df.drop_duplicates()
    df.reset_index(inplace=True, drop=True)

    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]
    print("y value counts: \n", str(y.value_counts()))
    print("y class ratio: 1:", str(round(y.value_counts()[0]/y.value_counts()[1])))

    cols = X.columns
    tar = y.name

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=rn)
    print("--- y train classes count: \n" + str(y_train.value_counts()))
    print("--- y train ratio: 1:" + str(round(y_train.value_counts()[0] / y_train.value_counts()[1])))
    print(" ")
    print("--- y test classes count: \n" + str(y_test.value_counts()))
    print("--- y test ratio: 1:" + str(round(y_test.value_counts()[0] / y_test.value_counts()[1])))

    ########## DAZZLE run ##########
    # DAZZLE oversampling run - DAZZLE oversampling technique
    print("----- DAZZLE1 -----")

    X_train_GAN = X_train.copy()
    y_train_GAN = y_train.copy()
    X_test_GAN = X_test.copy()
    y_test_GAN = y_test.copy()
    rt, X_train_new, y_train_new, X_test_scale = DAZZLE1Oversampling(X_train=X_train_GAN, 
                                                                     y_train=y_train_GAN, 
                                                                     X_test=X_test_GAN,
                                                                     y_test=y_test_GAN)

    X_train_new = pd.DataFrame(X_train_new, columns=cols)
    y_train_new = pd.Series(y_train_new, name=tar)
    X_test_scale = pd.DataFrame(X_test_scale, columns=cols)
    # scale data
    # scaler = StandardScaler()
    # X_train_scale = pd.DataFrame(scaler.fit_transform(X_train_new), columns=X_train_new.columns, index=X_train_new.index)
    # X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

    print("y train ratio: 1:" + str(round(y_train_new.value_counts()[0] / y_train_new.value_counts()[1])))

    # create models
    clf_SVM = SVC()
    clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
    clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
    clf_DT = DecisionTreeClassifier()
    clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
    clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
    clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
    clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

    clf_SVM.fit(X_train_new, y_train_new)
    clf_KNN.fit(X_train_new, y_train_new)
    clf_LR.fit(X_train_new, y_train_new)
    clf_DT.fit(X_train_new, y_train_new)
    clf_RF.fit(X_train_new, y_train_new)
    clf_LightGBM.fit(X_train_new, y_train_new)
    clf_Adaboost.fit(X_train_new, y_train_new)
    clf_GBDT.fit(X_train_new, y_train_new)

    y_pred_SVM = clf_SVM.predict(X_test_scale)
    y_pred_KNN = clf_KNN.predict(X_test_scale)
    y_pred_LR = clf_LR.predict(X_test_scale)
    y_pred_DT = clf_DT.predict(X_test_scale)
    y_pred_RF = clf_RF.predict(X_test_scale)
    y_pred_LightGBM = clf_LightGBM.predict(X_test_scale)
    y_pred_Adaboost = clf_Adaboost.predict(X_test_scale)
    y_pred_GBDT = clf_GBDT.predict(X_test_scale)

    with open(write_path, "a", newline="") as f:
        csv_writer = csv.writer(f)

        csv_writer.writerow(["DAZZLE1", rt, "SVM"] + evaluate_result(y_pred_SVM, y_test))
        csv_writer.writerow(["DAZZLE1", rt, "KNN"] + evaluate_result(y_pred_KNN, y_test))
        csv_writer.writerow(["DAZZLE1", rt, "LR"] + evaluate_result(y_pred_LR, y_test))
        csv_writer.writerow(["DAZZLE1", rt, "DT"] + evaluate_result(y_pred_DT, y_test))
        csv_writer.writerow(["DAZZLE1", rt, "RF"] + evaluate_result(y_pred_RF, y_test))
        csv_writer.writerow(["DAZZLE1", rt, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
        csv_writer.writerow(["DAZZLE1", rt, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
        csv_writer.writerow(["DAZZLE1", rt, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))

y value counts: 
 0    5367
1     904
Name: Vuln, dtype: int64
y class ratio: 1: 6
--- y train classes count: 
0    4293
1     723
Name: Vuln, dtype: int64
--- y train ratio: 1:6
 
--- y test classes count: 
0    1074
1     181
Name: Vuln, dtype: int64
--- y test ratio: 1:6
----- DAZZLE1 -----
[    0/121832] LG:-0.346 LD:-0.233 D:0.045 GP:0.020 RMSEAVG:0.171 NUM:0.171 SynTraiAuc:0.534 RFAcc:0.999                
[  500/121832] LG:-0.231 LD:0.237 D:0.282 GP:0.003 RMSEAVG:0.171 NUM:0.171 SynTraiAuc:0.534 RFAcc:0.999                 
[ 1000/121832] LG:-0.053 LD:0.181 D:0.241 GP:0.004 RMSEAVG:0.171 NUM:0.171 SynTraiAuc:0.534 RFAcc:0.999                 
[ 1500/121832] LG:-0.098 LD:0.124 D:0.131 GP:0.001 RMSEAVG:0.035 NUM:0.035 SynTraiAuc:0.534 RFAcc:0.994                 
[ 2000/121832] LG:0.069 LD:0.167 D:0.244 GP:0.005 RMSEAVG:0.035 NUM:0.035 SynTraiAuc:0.534 RFAcc:0.994                  
[ 2500/121832] LG:0.171 LD:0.238 D:0.306 GP:0.005 RMSEAVG:0.037 NUM:0.037 SynTraiAuc:0.534 RFAcc:0.9

[32500/121832] LG:-0.283 LD:-0.141 D:-0.104 GP:0.003 RMSEAVG:0.042 NUM:0.042 SynTraiAuc:0.656 RFAcc:0.996               
[33000/121832] LG:-0.352 LD:0.115 D:0.135 GP:0.001 RMSEAVG:0.042 NUM:0.042 SynTraiAuc:0.656 RFAcc:0.996                 
[33500/121832] LG:0.132 LD:0.163 D:0.194 GP:0.002 RMSEAVG:0.042 NUM:0.042 SynTraiAuc:0.656 RFAcc:0.996                  
[34000/121832] LG:0.362 LD:0.157 D:0.203 GP:0.003 RMSEAVG:0.035 NUM:0.035 SynTraiAuc:0.656 RFAcc:0.990                  
[34500/121832] LG:0.212 LD:0.227 D:0.248 GP:0.001 RMSEAVG:0.035 NUM:0.035 SynTraiAuc:0.656 RFAcc:0.990                  
[35000/121832] LG:0.296 LD:0.141 D:0.172 GP:0.002 RMSEAVG:0.060 NUM:0.060 SynTraiAuc:0.735 RFAcc:1.000                  
[35500/121832] LG:-0.129 LD:-0.322 D:-0.290 GP:0.002 RMSEAVG:0.060 NUM:0.060 SynTraiAuc:0.735 RFAcc:1.000               
[36000/121832] LG:-0.047 LD:-0.201 D:-0.172 GP:0.002 RMSEAVG:0.060 NUM:0.060 SynTraiAuc:0.735 RFAcc:1.000               
[36500/121832] LG:0.183 LD:-0.11

[66000/121832] LG:0.220 LD:-0.125 D:-0.113 GP:0.001 RMSEAVG:0.053 NUM:0.053 SynTraiAuc:0.558 RFAcc:1.000                
[66500/121832] LG:0.567 LD:-0.187 D:-0.102 GP:0.006 RMSEAVG:0.058 NUM:0.058 SynTraiAuc:0.558 RFAcc:1.000                
[67000/121832] LG:0.361 LD:-0.138 D:-0.101 GP:0.003 RMSEAVG:0.058 NUM:0.058 SynTraiAuc:0.558 RFAcc:1.000                
[67500/121832] LG:0.320 LD:-0.110 D:-0.081 GP:0.002 RMSEAVG:0.062 NUM:0.062 SynTraiAuc:0.558 RFAcc:1.000                
[68000/121832] LG:0.724 LD:-0.167 D:-0.062 GP:0.007 RMSEAVG:0.062 NUM:0.062 SynTraiAuc:0.558 RFAcc:1.000                
[68500/121832] LG:0.568 LD:-0.152 D:-0.096 GP:0.004 RMSEAVG:0.062 NUM:0.062 SynTraiAuc:0.558 RFAcc:1.000                
[69000/121832] LG:0.570 LD:0.044 D:0.156 GP:0.008 RMSEAVG:0.069 NUM:0.069 SynTraiAuc:0.558 RFAcc:0.994                  
[69500/121832] LG:0.611 LD:-0.064 D:-0.043 GP:0.001 RMSEAVG:0.069 NUM:0.069 SynTraiAuc:0.558 RFAcc:0.994                
[70000/121832] LG:0.471 LD:0.052

[99500/121832] LG:-3.236 LD:-0.167 D:-0.010 GP:0.011 RMSEAVG:0.056 NUM:0.056 SynTraiAuc:0.609 RFAcc:0.999               
[100000/121832] LG:-3.461 LD:-0.023 D:0.004 GP:0.002 RMSEAVG:0.059 NUM:0.059 SynTraiAuc:0.566 RFAcc:0.998               
[100500/121832] LG:-3.244 LD:-0.156 D:-0.115 GP:0.003 RMSEAVG:0.059 NUM:0.059 SynTraiAuc:0.566 RFAcc:0.998              
[101000/121832] LG:-3.240 LD:-0.111 D:-0.077 GP:0.002 RMSEAVG:0.059 NUM:0.059 SynTraiAuc:0.566 RFAcc:0.998              
[101500/121832] LG:-3.183 LD:-0.038 D:-0.008 GP:0.002 RMSEAVG:0.062 NUM:0.062 SynTraiAuc:0.566 RFAcc:0.999              
[102000/121832] LG:-3.205 LD:-0.134 D:-0.093 GP:0.003 RMSEAVG:0.062 NUM:0.062 SynTraiAuc:0.566 RFAcc:0.999              
[102500/121832] LG:-3.247 LD:-0.399 D:-0.209 GP:0.014 RMSEAVG:0.078 NUM:0.078 SynTraiAuc:0.566 RFAcc:1.000              
[103000/121832] LG:-3.138 LD:-0.088 D:-0.059 GP:0.002 RMSEAVG:0.078 NUM:0.078 SynTraiAuc:0.566 RFAcc:1.000              
[103500/121832] LG:-3.020 LD:-0.

[ 9500/70650] LG:0.516 LD:0.514 D:0.536 GP:0.003 RMSEAVG:0.023 NUM:0.023 SynTraiAuc:0.516 RFAcc:0.976                   
[10000/70650] LG:0.550 LD:0.403 D:0.441 GP:0.005 RMSEAVG:0.025 NUM:0.025 SynTraiAuc:0.606 RFAcc:0.984                   
[10500/70650] LG:0.559 LD:0.147 D:0.179 GP:0.004 RMSEAVG:0.025 NUM:0.025 SynTraiAuc:0.606 RFAcc:0.984                   
[11000/70650] LG:0.575 LD:0.260 D:0.293 GP:0.005 RMSEAVG:0.025 NUM:0.025 SynTraiAuc:0.606 RFAcc:0.984                   
[11500/70650] LG:0.625 LD:0.488 D:0.539 GP:0.007 RMSEAVG:0.026 NUM:0.026 SynTraiAuc:0.606 RFAcc:0.978                   
[12000/70650] LG:0.650 LD:0.489 D:0.537 GP:0.007 RMSEAVG:0.026 NUM:0.026 SynTraiAuc:0.606 RFAcc:0.978                   
[12500/70650] LG:0.698 LD:0.189 D:0.218 GP:0.004 RMSEAVG:0.024 NUM:0.024 SynTraiAuc:0.606 RFAcc:0.996                   
[13000/70650] LG:0.733 LD:0.229 D:0.262 GP:0.005 RMSEAVG:0.024 NUM:0.024 SynTraiAuc:0.606 RFAcc:0.996                   
[13500/70650] LG:0.751 LD:0.389 

...

[12000/28598] LG:0.580 LD:0.079 D:0.096 GP:0.002 RMSEAVG:0.008 NUM:0.008 SynTraiAuc:0.746 RFAcc:0.971                   
[12500/28598] LG:0.426 LD:0.064 D:0.079 GP:0.002 RMSEAVG:0.012 NUM:0.012 SynTraiAuc:0.746 RFAcc:0.961                   
[13000/28598] LG:0.486 LD:0.075 D:0.092 GP:0.002 RMSEAVG:0.012 NUM:0.012 SynTraiAuc:0.746 RFAcc:0.961                   
[13500/28598] LG:0.450 LD:0.051 D:0.068 GP:0.002 RMSEAVG:0.012 NUM:0.012 SynTraiAuc:0.746 RFAcc:0.961                   
[14000/28598] LG:0.492 LD:0.051 D:0.063 GP:0.001 RMSEAVG:0.012 NUM:0.012 SynTraiAuc:0.746 RFAcc:0.944                   
[14500/28598] LG:0.368 LD:0.089 D:0.097 GP:0.001 RMSEAVG:0.012 NUM:0.012 SynTraiAuc:0.746 RFAcc:0.944                   
[15000/28598] LG:0.436 LD:0.058 D:0.071 GP:0.002 RMSEAVG:0.010 NUM:0.010 SynTraiAuc:0.727 RFAcc:0.951                   
[15500/28598] LG:0.499 LD:0.068 D:0.090 GP:0.003 RMSEAVG:0.010 NUM:0.010 SynTraiAuc:0.727 RFAcc:0.951                   
[16000/28598] LG:0.474 LD:0.057 

[ 4000/37366] LG:0.790 LD:0.344 D:0.423 GP:0.009 RMSEAVG:0.078 NUM:0.078 SynTraiAuc:0.360 RFAcc:1.000                   
[ 4500/37366] LG:0.664 LD:0.139 D:0.148 GP:0.001 RMSEAVG:0.078 NUM:0.078 SynTraiAuc:0.360 RFAcc:1.000                   
[ 5000/37366] LG:0.521 LD:-0.042 D:0.233 GP:0.030 RMSEAVG:0.039 NUM:0.039 SynTraiAuc:0.453 RFAcc:1.000                  
[ 5500/37366] LG:0.693 LD:-0.217 D:0.160 GP:0.042 RMSEAVG:0.039 NUM:0.039 SynTraiAuc:0.453 RFAcc:1.000                  
[ 6000/37366] LG:0.322 LD:0.274 D:0.308 GP:0.004 RMSEAVG:0.039 NUM:0.039 SynTraiAuc:0.453 RFAcc:1.000                   
[ 6500/37366] LG:0.844 LD:0.110 D:0.181 GP:0.008 RMSEAVG:0.035 NUM:0.035 SynTraiAuc:0.453 RFAcc:0.990                   
[ 7000/37366] LG:0.792 LD:0.129 D:0.157 GP:0.003 RMSEAVG:0.035 NUM:0.035 SynTraiAuc:0.453 RFAcc:0.990                   
[ 7500/37366] LG:0.739 LD:0.072 D:0.176 GP:0.011 RMSEAVG:0.035 NUM:0.035 SynTraiAuc:0.453 RFAcc:1.000                   
[ 8000/37366] LG:0.925 LD:0.108 

[37366/37366] LG:1.538 LD:0.222 D:0.237 GP:0.002 RMSEAVG:0.027 NUM:0.027 SynTraiAuc:0.669 RFAcc:0.971                   
needed sample                                                                                                           
3570                                                                                                                    
[    0/57148] LG:-0.090 LD:-0.063 D:0.150 GP:0.030 RMSEAVG:0.234 NUM:0.234 SynTraiAuc:0.456 RFAcc:1.000                 
[  500/57148] LG:0.443 LD:0.353 D:0.404 GP:0.007 RMSEAVG:0.234 NUM:0.234 SynTraiAuc:0.456 RFAcc:1.000                   
[ 1000/57148] LG:0.481 LD:0.218 D:0.262 GP:0.006 RMSEAVG:0.234 NUM:0.234 SynTraiAuc:0.456 RFAcc:1.000                   
[ 1500/57148] LG:0.566 LD:0.282 D:0.338 GP:0.008 RMSEAVG:0.032 NUM:0.032 SynTraiAuc:0.456 RFAcc:0.991                   
[ 2000/57148] LG:0.605 LD:0.231 D:0.322 GP:0.013 RMSEAVG:0.032 NUM:0.032 SynTraiAuc:0.456 RFAcc:0.991                   
[ 2500/57148] LG:0.474 LD:0.133 

[32000/57148] LG:0.437 LD:0.044 D:0.070 GP:0.004 RMSEAVG:0.026 NUM:0.026 SynTraiAuc:0.542 RFAcc:0.980                   
[32500/57148] LG:0.364 LD:0.123 D:0.138 GP:0.002 RMSEAVG:0.018 NUM:0.018 SynTraiAuc:0.542 RFAcc:0.993                   
[33000/57148] LG:0.443 LD:0.012 D:0.031 GP:0.003 RMSEAVG:0.018 NUM:0.018 SynTraiAuc:0.542 RFAcc:0.993                   
[33500/57148] LG:0.450 LD:0.119 D:0.172 GP:0.008 RMSEAVG:0.018 NUM:0.018 SynTraiAuc:0.542 RFAcc:0.993                   
[34000/57148] LG:0.435 LD:0.075 D:0.095 GP:0.003 RMSEAVG:0.025 NUM:0.025 SynTraiAuc:0.542 RFAcc:0.959                   
[34500/57148] LG:0.497 LD:0.076 D:0.096 GP:0.003 RMSEAVG:0.025 NUM:0.025 SynTraiAuc:0.542 RFAcc:0.959                   
[35000/57148] LG:0.516 LD:0.078 D:0.099 GP:0.003 RMSEAVG:0.016 NUM:0.016 SynTraiAuc:0.635 RFAcc:0.984                   
[35500/57148] LG:0.437 LD:0.067 D:0.090 GP:0.003 RMSEAVG:0.016 NUM:0.016 SynTraiAuc:0.635 RFAcc:0.984                   
[36000/57148] LG:0.402 LD:0.124 

[ 6500/10360] LG:0.524 LD:0.253 D:0.276 GP:0.002 RMSEAVG:0.019 NUM:0.019 SynTraiAuc:0.727 RFAcc:0.970                   
[ 7000/10360] LG:0.561 LD:0.201 D:0.227 GP:0.003 RMSEAVG:0.019 NUM:0.019 SynTraiAuc:0.727 RFAcc:0.970                   
[ 7500/10360] LG:0.575 LD:0.212 D:0.239 GP:0.003 RMSEAVG:0.010 NUM:0.010 SynTraiAuc:0.727 RFAcc:0.920                   
[ 8000/10360] LG:0.565 LD:0.168 D:0.189 GP:0.002 RMSEAVG:0.010 NUM:0.010 SynTraiAuc:0.727 RFAcc:0.920                   
[ 8500/10360] LG:0.556 LD:0.204 D:0.226 GP:0.002 RMSEAVG:0.010 NUM:0.010 SynTraiAuc:0.727 RFAcc:0.920                   
[ 9000/10360] LG:0.594 LD:0.191 D:0.219 GP:0.003 RMSEAVG:0.009 NUM:0.009 SynTraiAuc:0.727 RFAcc:0.948                   
[ 9500/10360] LG:0.701 LD:0.179 D:0.202 GP:0.002 RMSEAVG:0.009 NUM:0.009 SynTraiAuc:0.727 RFAcc:0.948                   
[10000/10360] LG:0.723 LD:0.170 D:0.198 GP:0.003 RMSEAVG:0.009 NUM:0.009 SynTraiAuc:0.701 RFAcc:0.940                   
[10360/10360] LG:0.690 LD:0.128 

In [2]:
rnl = [35039, 25734, 217, 176, 372, 238, 480, 467, 310, 106]

for idx, rn in enumerate(rnl):
    write_file = f"JS_Vuln_res_r{idx+1}_rn{rn}.csv"
    write_path = f"{os.path.dirname(os.getcwd())}/result/JS_Vuln/{write_file}"
    data_path = f"{os.path.dirname(os.getcwd())}/data/JavaScript_Vulnerability/"
    datafiles = [f for f in os.listdir(data_path) if f.endswith("csv")]

    df = pd.read_csv(f"{data_path}/{datafiles[0]}")
    drop_columns = ["name", "longname", "path", "full_repo_path", "line", "column", "endline", "endcolumn"]
    df = df.drop(drop_columns, axis=1)
    df = df.drop_duplicates()
    df.reset_index(inplace=True, drop=True)

    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]
    print("y value counts: \n", str(y.value_counts()))
    print("y class ratio: 1:", str(round(y.value_counts()[0]/y.value_counts()[1])))

    cols = X.columns
    tar = y.name

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=rn)
    print("--- y train classes count: \n" + str(y_train.value_counts()))
    print("--- y train ratio: 1:" + str(round(y_train.value_counts()[0] / y_train.value_counts()[1])))
    print(" ")
    print("--- y test classes count: \n" + str(y_test.value_counts()))
    print("--- y test ratio: 1:" + str(round(y_test.value_counts()[0] / y_test.value_counts()[1])))

    ########## DAZZLE run ##########
    # DAZZLE oversampling run - DAZZLE oversampling technique
    print("----- DAZZLE2 -----")

    X_train_GAN = X_train.copy()
    y_train_GAN = y_train.copy()
    X_test_GAN = X_test.copy()
    y_test_GAN = y_test.copy()
    rt, X_train_new, y_train_new, X_test_scale = DAZZLE2Oversampling(X_train=X_train_GAN, 
                                                                     y_train=y_train_GAN, 
                                                                     X_test=X_test_GAN)

    X_train_new = pd.DataFrame(X_train_new, columns=cols)
    y_train_new = pd.Series(y_train_new, name=tar)
    X_test_scale = pd.DataFrame(X_test_scale, columns=cols)
    # scale data
    # scaler = StandardScaler()
    # X_train_scale = pd.DataFrame(scaler.fit_transform(X_train_new), columns=X_train_new.columns, index=X_train_new.index)
    # X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

    print("y train ratio: 1:" + str(round(y_train_new.value_counts()[0] / y_train_new.value_counts()[1])))

    # create models
    clf_SVM = SVC()
    clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
    clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
    clf_DT = DecisionTreeClassifier()
    clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
    clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
    clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
    clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

    clf_SVM.fit(X_train_new, y_train_new)
    clf_KNN.fit(X_train_new, y_train_new)
    clf_LR.fit(X_train_new, y_train_new)
    clf_DT.fit(X_train_new, y_train_new)
    clf_RF.fit(X_train_new, y_train_new)
    clf_LightGBM.fit(X_train_new, y_train_new)
    clf_Adaboost.fit(X_train_new, y_train_new)
    clf_GBDT.fit(X_train_new, y_train_new)

    y_pred_SVM = clf_SVM.predict(X_test_scale)
    y_pred_KNN = clf_KNN.predict(X_test_scale)
    y_pred_LR = clf_LR.predict(X_test_scale)
    y_pred_DT = clf_DT.predict(X_test_scale)
    y_pred_RF = clf_RF.predict(X_test_scale)
    y_pred_LightGBM = clf_LightGBM.predict(X_test_scale)
    y_pred_Adaboost = clf_Adaboost.predict(X_test_scale)
    y_pred_GBDT = clf_GBDT.predict(X_test_scale)

    with open(write_path, "a", newline="") as f:
        csv_writer = csv.writer(f)

        csv_writer.writerow(["DAZZLE2", rt, "SVM"] + evaluate_result(y_pred_SVM, y_test))
        csv_writer.writerow(["DAZZLE2", rt, "KNN"] + evaluate_result(y_pred_KNN, y_test))
        csv_writer.writerow(["DAZZLE2", rt, "LR"] + evaluate_result(y_pred_LR, y_test))
        csv_writer.writerow(["DAZZLE2", rt, "DT"] + evaluate_result(y_pred_DT, y_test))
        csv_writer.writerow(["DAZZLE2", rt, "RF"] + evaluate_result(y_pred_RF, y_test))
        csv_writer.writerow(["DAZZLE2", rt, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
        csv_writer.writerow(["DAZZLE2", rt, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
        csv_writer.writerow(["DAZZLE2", rt, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))

y value counts: 
 0    5367
1     904
Name: Vuln, dtype: int64
y class ratio: 1: 6
--- y train classes count: 
0    4293
1     723
Name: Vuln, dtype: int64
--- y train ratio: 1:6
 
--- y test classes count: 
0    1074
1     181
Name: Vuln, dtype: int64
--- y test ratio: 1:6
----- DAZZLE2 -----
[    0/65682] LG:-0.504 LD:-0.832 D:-0.334 GP:0.041 RMSEAVG:0.201 NUM:0.201 SynTraiAuc:0.366 RFAcc:0.998                
[  500/65682] LG:0.167 LD:1.754 D:1.851 GP:0.008 RMSEAVG:0.201 NUM:0.201 SynTraiAuc:0.366 RFAcc:0.998                   
[ 1000/65682] LG:-0.523 LD:0.402 D:0.448 GP:0.004 RMSEAVG:0.201 NUM:0.201 SynTraiAuc:0.366 RFAcc:0.998                  
[ 1500/65682] LG:-0.229 LD:0.242 D:0.326 GP:0.007 RMSEAVG:0.035 NUM:0.035 SynTraiAuc:0.366 RFAcc:0.994                  
[ 2000/65682] LG:-0.055 LD:0.046 D:0.105 GP:0.005 RMSEAVG:0.035 NUM:0.035 SynTraiAuc:0.366 RFAcc:0.994                  
[ 2500/65682] LG:-0.132 LD:0.051 D:0.096 GP:0.004 RMSEAVG:0.027 NUM:0.027 SynTraiAuc:0.366 RFAcc:0.9

[32500/65682] LG:0.723 LD:0.067 D:0.100 GP:0.003 RMSEAVG:0.011 NUM:0.011 SynTraiAuc:0.725 RFAcc:0.966                   
[33000/65682] LG:0.689 LD:0.219 D:0.244 GP:0.002 RMSEAVG:0.011 NUM:0.011 SynTraiAuc:0.725 RFAcc:0.966                   
[33500/65682] LG:0.699 LD:0.015 D:0.020 GP:0.000 RMSEAVG:0.011 NUM:0.011 SynTraiAuc:0.725 RFAcc:0.966                   
[34000/65682] LG:0.787 LD:0.107 D:0.134 GP:0.002 RMSEAVG:0.014 NUM:0.014 SynTraiAuc:0.725 RFAcc:0.979                   
[34500/65682] LG:0.756 LD:0.062 D:0.084 GP:0.002 RMSEAVG:0.014 NUM:0.014 SynTraiAuc:0.725 RFAcc:0.979                   
[35000/65682] LG:0.688 LD:-0.026 D:-0.001 GP:0.002 RMSEAVG:0.011 NUM:0.011 SynTraiAuc:0.743 RFAcc:0.979                 
[35500/65682] LG:0.705 LD:0.052 D:0.069 GP:0.001 RMSEAVG:0.011 NUM:0.011 SynTraiAuc:0.743 RFAcc:0.979                   
[36000/65682] LG:0.724 LD:0.173 D:0.213 GP:0.003 RMSEAVG:0.011 NUM:0.011 SynTraiAuc:0.743 RFAcc:0.979                   
[36500/65682] LG:0.773 LD:0.030 

[65682/65682] LG:0.978 LD:0.118 D:0.132 GP:0.001 RMSEAVG:0.015 NUM:0.015 SynTraiAuc:0.701 RFAcc:0.956                   
needed sample                                                                                                           
3037                                                                                                                    
[    0/9112] LG:-0.536 LD:-0.459 D:-0.278 GP:0.012 RMSEAVG:0.322 NUM:0.322 SynTraiAuc:0.469 RFAcc:1.000                 
[  500/9112] LG:0.547 LD:1.265 D:1.286 GP:0.001 RMSEAVG:0.322 NUM:0.322 SynTraiAuc:0.469 RFAcc:1.000                    
[ 1000/9112] LG:0.401 LD:0.848 D:0.860 GP:0.001 RMSEAVG:0.322 NUM:0.322 SynTraiAuc:0.469 RFAcc:1.000                    
[ 1500/9112] LG:0.212 LD:0.483 D:0.494 GP:0.001 RMSEAVG:0.096 NUM:0.096 SynTraiAuc:0.469 RFAcc:1.000                    
[ 2000/9112] LG:0.100 LD:0.368 D:0.394 GP:0.002 RMSEAVG:0.096 NUM:0.096 SynTraiAuc:0.469 RFAcc:1.000                    
[ 2500/9112] LG:0.169 LD:0.260 D

[ 8000/19564] LG:1.231 LD:0.318 D:0.352 GP:0.004 RMSEAVG:0.022 NUM:0.022 SynTraiAuc:0.551 RFAcc:0.958                   
[ 8500/19564] LG:1.244 LD:0.301 D:0.324 GP:0.003 RMSEAVG:0.022 NUM:0.022 SynTraiAuc:0.551 RFAcc:0.958                   
[ 9000/19564] LG:1.290 LD:0.291 D:0.309 GP:0.002 RMSEAVG:0.023 NUM:0.023 SynTraiAuc:0.551 RFAcc:0.958                   
[ 9500/19564] LG:1.290 LD:0.258 D:0.287 GP:0.003 RMSEAVG:0.023 NUM:0.023 SynTraiAuc:0.551 RFAcc:0.958                   
[10000/19564] LG:1.308 LD:0.244 D:0.267 GP:0.003 RMSEAVG:0.019 NUM:0.019 SynTraiAuc:0.664 RFAcc:0.975                   
[10500/19564] LG:1.271 LD:0.270 D:0.312 GP:0.005 RMSEAVG:0.019 NUM:0.019 SynTraiAuc:0.664 RFAcc:0.975                   
[11000/19564] LG:1.289 LD:0.235 D:0.258 GP:0.003 RMSEAVG:0.019 NUM:0.019 SynTraiAuc:0.664 RFAcc:0.975                   
[11500/19564] LG:1.303 LD:0.210 D:0.227 GP:0.002 RMSEAVG:0.017 NUM:0.017 SynTraiAuc:0.664 RFAcc:0.975                   
[12000/19564] LG:1.287 LD:0.346 

[20000/84105] LG:-1.121 LD:0.072 D:0.089 GP:0.001 RMSEAVG:0.014 NUM:0.014 SynTraiAuc:0.691 RFAcc:0.960                  
[20500/84105] LG:-1.150 LD:0.020 D:0.041 GP:0.001 RMSEAVG:0.014 NUM:0.014 SynTraiAuc:0.691 RFAcc:0.960                  
[21000/84105] LG:-1.225 LD:0.115 D:0.136 GP:0.001 RMSEAVG:0.014 NUM:0.014 SynTraiAuc:0.691 RFAcc:0.960                  
[21500/84105] LG:-1.165 LD:0.109 D:0.163 GP:0.003 RMSEAVG:0.012 NUM:0.012 SynTraiAuc:0.691 RFAcc:0.971                  
[22000/84105] LG:-1.227 LD:0.186 D:0.213 GP:0.002 RMSEAVG:0.012 NUM:0.012 SynTraiAuc:0.691 RFAcc:0.971                  
[22500/84105] LG:-1.210 LD:0.030 D:0.067 GP:0.002 RMSEAVG:0.015 NUM:0.015 SynTraiAuc:0.691 RFAcc:0.945                  
[23000/84105] LG:-1.247 LD:0.064 D:0.104 GP:0.002 RMSEAVG:0.015 NUM:0.015 SynTraiAuc:0.691 RFAcc:0.945                  
[23500/84105] LG:-1.169 LD:0.113 D:0.139 GP:0.001 RMSEAVG:0.015 NUM:0.015 SynTraiAuc:0.691 RFAcc:0.945                  
[24000/84105] LG:-1.200 LD:0.183

...

[63500/101460] LG:-0.113 LD:0.172 D:0.209 GP:0.004 RMSEAVG:0.028 NUM:0.028 SynTraiAuc:0.686 RFAcc:0.991                 
[64000/101460] LG:-0.341 LD:0.008 D:0.033 GP:0.003 RMSEAVG:0.039 NUM:0.039 SynTraiAuc:0.686 RFAcc:0.996                 
[64500/101460] LG:-0.232 LD:-0.074 D:-0.024 GP:0.006 RMSEAVG:0.039 NUM:0.039 SynTraiAuc:0.686 RFAcc:0.996               
[65000/101460] LG:-0.266 LD:-0.090 D:-0.076 GP:0.002 RMSEAVG:0.031 NUM:0.031 SynTraiAuc:0.641 RFAcc:0.975               
[65500/101460] LG:-0.031 LD:0.018 D:0.028 GP:0.001 RMSEAVG:0.031 NUM:0.031 SynTraiAuc:0.641 RFAcc:0.975                 
[66000/101460] LG:-0.142 LD:0.092 D:0.112 GP:0.002 RMSEAVG:0.031 NUM:0.031 SynTraiAuc:0.641 RFAcc:0.975                 
[66500/101460] LG:-0.221 LD:0.018 D:0.121 GP:0.011 RMSEAVG:0.028 NUM:0.028 SynTraiAuc:0.641 RFAcc:0.998                 
[67000/101460] LG:-0.296 LD:0.037 D:0.055 GP:0.002 RMSEAVG:0.028 NUM:0.028 SynTraiAuc:0.641 RFAcc:0.998                 
[67500/101460] LG:-0.233 LD:-0.1

[97000/101460] LG:-0.258 LD:-0.060 D:-0.039 GP:0.002 RMSEAVG:0.050 NUM:0.050 SynTraiAuc:0.612 RFAcc:1.000               
[97500/101460] LG:-0.093 LD:0.091 D:0.100 GP:0.001 RMSEAVG:0.037 NUM:0.037 SynTraiAuc:0.612 RFAcc:0.998                 
[98000/101460] LG:-0.393 LD:0.061 D:0.088 GP:0.003 RMSEAVG:0.037 NUM:0.037 SynTraiAuc:0.612 RFAcc:0.998                 
[98500/101460] LG:-0.322 LD:0.016 D:0.031 GP:0.002 RMSEAVG:0.037 NUM:0.037 SynTraiAuc:0.612 RFAcc:0.998                 
[99000/101460] LG:-0.404 LD:-0.113 D:-0.100 GP:0.001 RMSEAVG:0.034 NUM:0.034 SynTraiAuc:0.612 RFAcc:0.990               
[99500/101460] LG:-0.237 LD:-0.012 D:0.012 GP:0.003 RMSEAVG:0.034 NUM:0.034 SynTraiAuc:0.612 RFAcc:0.990                
[100000/101460] LG:-0.139 LD:0.191 D:0.229 GP:0.004 RMSEAVG:0.027 NUM:0.027 SynTraiAuc:0.698 RFAcc:0.991                
[100500/101460] LG:-0.386 LD:0.084 D:0.098 GP:0.002 RMSEAVG:0.027 NUM:0.027 SynTraiAuc:0.698 RFAcc:0.991                
[101000/101460] LG:-0.542 LD:-0.

[ 7000/8710] LG:0.390 LD:0.126 D:0.146 GP:0.001 RMSEAVG:0.147 NUM:0.147 SynTraiAuc:0.589 RFAcc:0.998                    
[ 7500/8710] LG:0.453 LD:0.032 D:0.130 GP:0.005 RMSEAVG:0.180 NUM:0.180 SynTraiAuc:0.589 RFAcc:0.996                    
[ 8000/8710] LG:0.122 LD:-0.337 D:-0.200 GP:0.008 RMSEAVG:0.180 NUM:0.180 SynTraiAuc:0.589 RFAcc:0.996                  
[ 8500/8710] LG:0.513 LD:-0.090 D:0.025 GP:0.006 RMSEAVG:0.180 NUM:0.180 SynTraiAuc:0.589 RFAcc:0.996                   
[ 8710/8710] LG:0.479 LD:0.225 D:0.386 GP:0.009 RMSEAVG:0.180 NUM:0.180 SynTraiAuc:0.589 RFAcc:0.996                    
needed sample                                                                                                           
2989                                                                                                                    
[    0/49929] LG:0.030 LD:-2.967 D:-0.144 GP:0.166 RMSEAVG:0.204 NUM:0.204 SynTraiAuc:0.518 RFAcc:1.000                 
[  500/49929] LG:0.564 LD:0.573 

[30000/49929] LG:0.895 LD:0.032 D:0.042 GP:0.001 RMSEAVG:0.030 NUM:0.030 SynTraiAuc:0.655 RFAcc:0.974                   
[30500/49929] LG:0.773 LD:-0.048 D:0.032 GP:0.005 RMSEAVG:0.030 NUM:0.030 SynTraiAuc:0.655 RFAcc:0.974                  
[31000/49929] LG:0.811 LD:-0.109 D:-0.068 GP:0.002 RMSEAVG:0.030 NUM:0.030 SynTraiAuc:0.655 RFAcc:0.974                 
[31500/49929] LG:0.713 LD:-0.070 D:-0.045 GP:0.002 RMSEAVG:0.026 NUM:0.026 SynTraiAuc:0.655 RFAcc:0.998                 
[32000/49929] LG:0.752 LD:-0.047 D:0.006 GP:0.003 RMSEAVG:0.026 NUM:0.026 SynTraiAuc:0.655 RFAcc:0.998                  
[32500/49929] LG:0.831 LD:-0.046 D:-0.028 GP:0.001 RMSEAVG:0.049 NUM:0.049 SynTraiAuc:0.655 RFAcc:0.999                 
[33000/49929] LG:0.691 LD:-0.098 D:-0.031 GP:0.004 RMSEAVG:0.049 NUM:0.049 SynTraiAuc:0.655 RFAcc:0.999                 
[33500/49929] LG:0.544 LD:-0.123 D:-0.066 GP:0.003 RMSEAVG:0.049 NUM:0.049 SynTraiAuc:0.655 RFAcc:0.999                 
[34000/49929] LG:0.714 LD:-0.110

[12000/21574] LG:0.722 LD:0.081 D:0.116 GP:0.002 RMSEAVG:0.010 NUM:0.010 SynTraiAuc:0.696 RFAcc:0.940                   
[12500/21574] LG:0.681 LD:0.093 D:0.114 GP:0.001 RMSEAVG:0.011 NUM:0.011 SynTraiAuc:0.696 RFAcc:0.956                   
[13000/21574] LG:0.690 LD:0.121 D:0.144 GP:0.001 RMSEAVG:0.011 NUM:0.011 SynTraiAuc:0.696 RFAcc:0.956                   
[13500/21574] LG:0.698 LD:0.068 D:0.092 GP:0.001 RMSEAVG:0.011 NUM:0.011 SynTraiAuc:0.696 RFAcc:0.956                   
[14000/21574] LG:0.736 LD:0.073 D:0.088 GP:0.001 RMSEAVG:0.011 NUM:0.011 SynTraiAuc:0.696 RFAcc:0.955                   
[14500/21574] LG:0.711 LD:0.149 D:0.187 GP:0.002 RMSEAVG:0.011 NUM:0.011 SynTraiAuc:0.696 RFAcc:0.955                   
[15000/21574] LG:0.729 LD:0.036 D:0.065 GP:0.001 RMSEAVG:0.011 NUM:0.011 SynTraiAuc:0.686 RFAcc:0.968                   
[15500/21574] LG:0.682 LD:0.076 D:0.100 GP:0.001 RMSEAVG:0.011 NUM:0.011 SynTraiAuc:0.686 RFAcc:0.968                   
[16000/21574] LG:0.691 LD:0.072 